
# ProteinDT (Hugging Face) — Download & Quick Check (Colab)

- basic statistics
- Testing downloading API (experimental & alphafoldv2)
- Generate CoT (Using GPT) with/without groud truth (Testing & Debugging)


In [ ]:

#@title Install dependencies
!pip -q install --upgrade huggingface_hub datasets pandas biopython lmdb tqdm rich


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.3/563.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 15.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.2 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.2 which is incomp

In [ ]:
#@title Mount Google Drive
USE_DRIVE = True  #@param {type:"boolean"}
DRIVE_SUBDIR = "hf/proteinDT"  #@param {type:"string"}
BASE_DIR = ""


if USE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    from pathlib import Path as _Path
    _drive_base = _Path('')
    BASE_DIR = _drive_base / DRIVE_SUBDIR
    #c.mkdir(parents=True, exist_ok=True)
    print(f"Using Google Drive folder as BASE_DIR: {BASE_DIR}")
else:
    print(f"Using local runtime folder as BASE_DIR: {BASE_DIR}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using Google Drive folder as BASE_DIR: /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT


In [ ]:
from pathlib import Path
from huggingface_hub import snapshot_download
import os, json, lmdb, pickle, pandas as pd
from tqdm import tqdm
from rich import print as rprint

In [ ]:

REPO_ID = "chao1224/ProteinDT"
REPO_TYPE = "dataset"


DOWNLOAD_BIG_ZIP = False
DOWNLOAD_CHECKPOINTS = False

allow_patterns = [
    "SwissProtCLAP/*",
    "downstream_Editing/datasets_and_checkpoints/**",
    "*.md",
    ".gitattributes",
]
if DOWNLOAD_BIG_ZIP:
    allow_patterns.append("downstream_datasets.zip")

ignore_patterns = []
if not DOWNLOAD_CHECKPOINTS:
    ignore_patterns.append("**/*.pth")

BASE_DIR.mkdir(parents=True, exist_ok=True)
rprint(f"[bold]Will download to:[/bold] {BASE_DIR}")
rprint(f"[bold]Allow patterns:[/bold] {allow_patterns}")
rprint(f"[bold]Ignore patterns:[/bold] {ignore_patterns}")


Will download to: /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT

Allow patterns: ['SwissProtCLAP/*', 'downstream_Editing/datasets_and_checkpoints/**', '*.md', '.gitattributes']

Ignore patterns: ['**/*.pth']

In [ ]:

local_dir = snapshot_download(
    repo_id=REPO_ID,
    repo_type=REPO_TYPE,
    local_dir=str(BASE_DIR),
    local_dir_use_symlinks=False,
    allow_patterns=allow_patterns,
    ignore_patterns=ignore_patterns,
)
rprint(f"[green]✓ Downloaded snapshot to:[/green] {local_dir}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:982: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 36 files:   0%|          | 0/36 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/20.0 [00:00<?, ?B/s]

SwissProtCLAP/text_sequence.txt:   0%|          | 0.00/299M [00:00<?, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

...

lock.mdb:   0%|          | 0.00/8.19k [00:00<?, ?B/s]

downstream_Editing/datasets_and_checkpoi(…):   0%|          | 0.00/93.7M [00:00<?, ?B/s]

lock.mdb:   0%|          | 0.00/8.19k [00:00<?, ?B/s]

downstream_Editing/datasets_and_checkpoi(…):   0%|          | 0.00/305M [00:00<?, ?B/s]

✓ Downloaded snapshot to: /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT

In [ ]:

#@title Show directory tree & sizes
import humanize

def iter_files(root):
    for p in Path(root).rglob("*"):
        if p.is_file():
            yield p

total_bytes = 0
rows = []
for p in iter_files(BASE_DIR):
    try:
        sz = p.stat().st_size
        total_bytes += sz
        rows.append((str(p.relative_to(BASE_DIR)), sz))
    except Exception as e:
        rows.append((str(p.relative_to(BASE_DIR)), None))

# Print a concise table
print(f"Total files: {len(rows)}")
print(f"Total size: {humanize.naturalsize(total_bytes)}")

# Show the largest 25 files
top = sorted(rows, key=lambda x: (x[1] or 0), reverse=True)[:25]
print("\nTop 25 largest files:")
for rel, sz in top:
    print(f"{rel:100s}  {humanize.naturalsize(sz) if sz is not None else 'N/A'}")


Total files: 1113
Total size: 5.3 GB

Top 25 largest files:
downstream_Editing/datasets_and_checkpoints/stability/oracle/pytorch_model_stability.bin              1.7 GB
downstream_Editing/datasets_and_checkpoints/structure/oracle/pytorch_model_ss3.bin                    1.7 GB
downstream_Editing/datasets_and_checkpoints/structure/secondary_structure_train.lmdb/data.mdb         373.8 MB
downstream_Editing/datasets_and_checkpoints/uniprot_sprot_xml_text.txt                                304.6 MB
SwissProtCLAP/text_sequence.txt                                                                       298.9 MB
SwissProtCLAP/protein_sequence.txt                                                                    201.8 MB
downstream_Editing/datasets_and_checkpoints/structure/secondary_structure_valid.lmdb/data.mdb         93.7 MB
subset_1k/downloads/O15287/7KZQ.cif                                                                   33.2 MB
downstream_Editing/datasets_and_checkpoints/structure/seco

In [ ]:

#@title quick dataset counts
sp_dir = BASE_DIR / "SwissProtCLAP"
seq_path = sp_dir / "protein_sequence.txt"
txt_path = sp_dir / "text_sequence.txt"

def safe_count_lines(path, limit=None):
    n = 0
    head = []
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            if limit is not None and n < limit:
                head.append(line.rstrip("\n"))
            n += 1
    return n, head

if seq_path.exists() and txt_path.exists():
    n_seq, head_seq = safe_count_lines(seq_path, limit=5)
    n_txt, head_txt = safe_count_lines(txt_path, limit=5)
    print(f"protein_sequence.txt — lines: {n_seq}")
    print(f"text_sequence.txt    — lines: {n_txt}")
    print("\nFirst 5 protein sequences:")
    for i, s in enumerate(head_seq, 1):
        print(f"[{i}] {s[:120]}{'...' if len(s)>120 else ''}")
    print("\nFirst 5 text descriptions:")
    for i, s in enumerate(head_txt, 1):
        print(f"[{i}] {s[:120]}{'...' if len(s)>120 else ''}")
    if n_seq == n_txt:
        print(f"\n[OK] The files appear line-aligned (same number of lines: {n_seq}).")
    else:
        print(f"\n[WARN] Line counts differ: sequences={n_seq}, texts={n_txt}.")
else:
    print("[WARN] SwissProtCLAP files not found (were they excluded by patterns?).")


protein_sequence.txt — lines: 1082318
text_sequence.txt    — lines: 1082318

First 5 protein sequences:
[1] P0C9F0
[2] MVRLFYNPIKYLFYRRSCKKRLRKALKKLNFYHPPKECCQIYRLLENAPGGTYFITENMTNELIMIAKDPVDKKIKSVKLYLTGNYIKINQHYYINIYMYLMRYNQIYKYPLICFSKYSK...
[3] P0C9F1
[4] MVRLFHNPIKCLFYRGSRKTREKKLRKSLKKLNFYHPPGDCCQIYRLLENVPGGTYFITENMTNELIMIVKDSVDKKIKSVKLNFYGSYIKIHQHYYINIYMYLMRYTQIYKYPLICFNK...
[5] P0C9F2

First 5 text descriptions:
[1] P0C9F0
[2] Plays a role in virus cell tropism, and may be required for efficient virus replication in macrophages. Belongs to the a...
[3] P0C9F1
[4] Plays a role in virus cell tropism, and may be required for efficient virus replication in macrophages. Belongs to the a...
[5] P0C9F2

[OK] The files appear line-aligned (same number of lines: 1082318).


In [ ]:

#@title Stability dataset: peek JSON (train/valid/test)
import json, statistics

stab_dir = BASE_DIR / "downstream_Editing" / "datasets_and_checkpoints" / "stability"
js_files = ["stability_train.json", "stability_valid.json", "stability_test.json"]
for jf in js_files:
    p = stab_dir / jf
    if not p.exists():
        print(f"[WARN] Missing {p}")
        continue
    data = json.loads(p.read_text())
    print(f"\nFile: {jf} — entries: {len(data)}")
    # show one example with keys
    ex = data[0]
    print(f"Keys: {list(ex.keys())}")
    # if sequences present, show stats
    seq_lens = []
    for d in data[:10000]:
        s = d.get('sequence') or d.get('protein') or d.get('mutated_seq') or d.get('seq')
        if isinstance(s, str):
            seq_lens.append(len(s))
    if seq_lens:
        print(f"Example sequence length stats (first {len(seq_lens)}): "
              f"min={min(seq_lens)}, max={max(seq_lens)}, mean={sum(seq_lens)/len(seq_lens):.1f}")
    # show first example for inspection
    print("Example record (truncated):")
    print({k: (v if isinstance(v, (int, float)) else str(v)[:120] + ('...' if len(str(v))>120 else '')) for k, v in ex.items()})



File: stability_train.json — entries: 53614
Keys: ['id', 'primary', 'protein_length', 'topology', 'parent', 'stability_score']
Example record (truncated):
{'id': 'EEHEE_rd1_0001.pdb', 'primary': 'GSQEVNSGTQTYKNASPEEAERIARKAGATTWTEKGNKWEIRI', 'protein_length': 43, 'topology': 'EEHEE', 'parent': 'EEHEE_rd1_0001', 'stability_score': '[0.17000000178813934]'}

File: stability_valid.json — entries: 2512
Keys: ['id', 'primary', 'protein_length', 'topology', 'parent', 'stability_score']
Example record (truncated):
{'id': 'EEHEE_rd4_0008.pdb', 'primary': 'ITVDVNGVTYHFDNPEEAYKFVVRVARELNLTYEWHGNKVRVELESSGGS', 'protein_length': 50, 'topology': 'EEHEE', 'parent': 'EEHEE_rd4_0008', 'stability_score': '[1.4500000476837158]'}

File: stability_test.json — entries: 12851
Keys: ['id', 'primary', 'protein_length', 'topology', 'parent', 'stability_score']
Example record (truncated):
{'id': 'EEHEE_rd3_0037.pdb', 'primary': 'TTIKVNGQEYTVPLSPEQAAKAAKKRWPDYEVQIHGNTVKVTR', 'protein_length': 43, 'topology': 'EE

In [ ]:

#@title Region dataset: CSV overview
reg_dir = BASE_DIR / "downstream_Editing" / "datasets_and_checkpoints" / "region"

for fname in ["raw_sequence.csv", "preprocessed_data.csv", "PDBvsAF2-Table1.csv", "TdarkAF2-Table1.csv"]:
    p = reg_dir / fname
    if p.exists():
        df = pd.read_csv(p)
        print(f"\n[{fname}] shape={df.shape}")
        print("Columns:", list(df.columns))
        print(df.head(3).to_string(index=False))
    else:
        print(f"[WARN] Missing {p}")



[raw_sequence.csv] shape=(1810, 2)
Columns: ['uniprot', 'AA']
   uniprot                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          AA
A0A1D8PJF9 MISRNLRTSSTKTFYGSILRSQIRSISKNNSQQATAQSTSRPQPSPAFNTADTSTQPIINDPTLNKHQSSAISRKKKEQLMDDSFIGLTGGEIFHEMMLRHKVDTVFGYAGGAILPVFDAIYNSDKFKFVLPRHEQGAGHMAEGYARASGKPGVVLVTSGPGATNVITPMADALMDGVPLVVFSGQVPTTAIGTDAFQEADIVGISRSCTKWNVMVKNVAELPRRINEAFE

In [ ]:

#@title Peptide binding (MISATO) files
misato_dir = BASE_DIR / "downstream_Editing" / "datasets_and_checkpoints" / "peptide_binding" / "MISATO"

json_path = misato_dir / "PDB_idx2text.json"
txt_map = misato_dir / "PDB_mapping_data.txt"
csv_pre = misato_dir / "preprocessed_data.csv"

if json_path.exists():
    with open(json_path, "r") as f:
        js = json.load(f)
    print(f"PDB_idx2text.json — entries: {len(js)}")
    # show 3 (id -> text) pairs
    for i, (k, v) in enumerate(list(js.items())[:3], 1):
        print(f" {i}. {k} → {str(v)[:100]}{'...' if len(str(v))>100 else ''}")
else:
    print("[WARN] PDB_idx2text.json not found")

if txt_map.exists():
    print("\nPDB_mapping_data.txt — first 5 lines:")
    with open(txt_map, "r") as f:
        for i, line in enumerate(f):
            print(line.strip())
            if i >= 4: break
else:
    print("[WARN] PDB_mapping_data.txt not found")

if csv_pre.exists():
    df = pd.read_csv(csv_pre)
    print(f"\npreprocessed_data.csv — shape={df.shape}")
    print(df.head(3).to_string(index=False))
else:
    print("[INFO] preprocessed_data.csv not present (expected in some setups).")


PDB_idx2text.json — entries: 124
 1. 2PEM → An RbcL-specific chaperone. Required for assembly of the RbcL8 core, acting downstream of the major ...
 2. 6EGW → Phosphorylates a large number of substrates in the cytoplasm and the nucleus (By similarity). Phosph...
 3. 6EH2 → Phosphorylates a large number of substrates in the cytoplasm and the nucleus (By similarity). Phosph...

PDB_mapping_data.txt — first 5 lines:
3O0E,SGGDGRGXNTGAXST,AEIYNKDGNKVDLYGKAVGLXYFSKGNGENSYGGNGDMTYARLGFKGETQINSDLTGYGQWEYNFQGNNSEGADAQTGNKTRLAFAGLKYADVGSFDYGRNYGVVYDALGYTDMLPEFGGDTAYSDDFFVGRVGGVATYRNSNFFGLVDGLNFAVQYLGKNERDTARRSNGDGVGGSISYEYEGFGIVGAYGAADRTNLQEAQPLGNGKKAEQWATGLKYDANNIYLAANYGETRNATPITNKFTNTSGFANKTQDVLLVAQYQFDFGLRPSIAYTKSKAKDVEGIGDVDLVNYFEVGATYYFNKNMSTYVDYIINQIDSDNKLGVGSDDTVAVGIVYQFAEIYNKDGNKVDLYGKAVGLXYFSKGNGENSYGGNGDMTYARLGFKGETQINSDLTGYGQWEYNFQGNNSEGADAQTGNKTRLAFAGLKYADVGSFDYGRNYGVVYDALGYTDMLPEFGGDTAYSDDFFVGRVGGVATYRNSNFFGLVDGLNFAVQYLGKNERDTARRSNGDGVGGSISYEYEGFGIVGAYGAADRTNLQEAQPLGNGKKAEQWATGLKYDA

In [ ]:

#@title Structure dataset (LMDB): count entries
str_dir = BASE_DIR / "downstream_Editing" / "datasets_and_checkpoints" / "structure"
lmdb_files = [
    "secondary_structure_train.lmdb",
    "secondary_structure_valid.lmdb",
    "secondary_structure_ts115.lmdb",
    "secondary_structure_casp12.lmdb",
    "secondary_structure_cb513.lmdb",
]

def count_lmdb(path: Path, max_preview: int = 3):
    if not path.exists():
        print(f"[WARN] {path.name} missing")
        return
    env = lmdb.open(str(path), readonly=True, lock=False, readahead=False, max_readers=1024)
    with env.begin(write=False) as txn:
        c = txn.cursor()
        n = 0
        print(f"\nLMDB: {path.name}")
        for k, v in c:
            n += 1
            if n <= max_preview:
                # Try to unpickle or just show sizes
                try:
                    obj = pickle.loads(v)
                    preview = str(type(obj))
                except Exception:
                    preview = f"{len(v)} bytes"
                print(f"  key={k[:30]!r} → {preview}")
        print(f"  total entries (approx): {n}")
    env.close()

for name in lmdb_files:
    count_lmdb(str_dir / name)



LMDB: secondary_structure_train.lmdb
  key=b'0' → <class 'dict'>
  key=b'1' → <class 'dict'>
  key=b'10' → <class 'dict'>
  total entries (approx): 8679

LMDB: secondary_structure_valid.lmdb
  key=b'0' → <class 'dict'>
  key=b'1' → <class 'dict'>
  key=b'10' → <class 'dict'>
  total entries (approx): 2171

LMDB: secondary_structure_ts115.lmdb
  key=b'0' → <class 'dict'>
  key=b'1' → <class 'dict'>
  key=b'10' → <class 'dict'>
  total entries (approx): 116

LMDB: secondary_structure_casp12.lmdb
  key=b'0' → <class 'dict'>
  key=b'1' → <class 'dict'>
  key=b'10' → <class 'dict'>
  total entries (approx): 22

LMDB: secondary_structure_cb513.lmdb
  key=b'0' → <class 'dict'>
  key=b'1' → <class 'dict'>
  key=b'10' → <class 'dict'>
  total entries (approx): 514


### Test the API

In [ ]:
#@title Download AF2 predicted PDB/CIF and experimental mmCIFs (RCSB) by UniProt ID
import re, os, sys, time, json, math, shutil, requests
from pathlib import Path
import pandas as pd

# Requires BASE_DIR from the earlier config cell
REGION_DIR = BASE_DIR / "downstream_Editing" / "datasets_and_checkpoints" / "region"

def _safe_filename(s: str) -> str:
    return re.sub(r'[^0-9A-Za-z._-]+', '_', s)

def load_pdbvsaf2_table():
    # Load once and cache in global
    global _pdbvsaf2_df
    if '_pdbvsaf2_df' in globals() and _pdbvsaf2_df is not None:
        return _pdbvsaf2_df
    p = REGION_DIR / "PDBvsAF2-Table1.csv"
    if not p.exists():
        print(f"[WARN] {p} not found. Only AF2 download by UniProt will be attempted.")
        _pdbvsaf2_df = None
        return _pdbvsaf2_df
    _pdbvsaf2_df = pd.read_csv(p)
    return _pdbvsaf2_df

def find_rows_for_uniprot(uniprot_id: str):
    df = load_pdbvsaf2_table()
    if df is None:
        return None
    uni_cols = [c for c in df.columns if c.lower().startswith("uniprot")]
    if not uni_cols:
        return None
    ucol = uni_cols[0]
    rows = df[df[ucol].astype(str) == uniprot_id]
    if rows.empty:
        return None
    return rows

def guess_af2_model_filename(uniprot_id: str, model_field: str | None) -> list[str]:
    """Build a list of plausible AFDB filenames to try, ordered by likelihood."""
    candidates = []
    # Prefer model_field's version if provided (e.g., 'AF-A0A1...-F1-model_v1')
    if model_field and isinstance(model_field, str):
        m = re.search(r'(AF-[A-Za-z0-9]+-F1-model_v\\d+)', model_field)
        if m:
            candidates.append(m.group(1))
    # Common current versions on AFDB
    base = f"AF-{uniprot_id}-F1-model_v{{ver}}"
    for ver in ["4","3","2","1"]:
        fname = base.format(ver=ver)
        if fname not in candidates:
            candidates.append(fname)
    return candidates

def try_download(url: str, out_path: Path, timeout=60):
    try:
        r = requests.get(url, stream=True, timeout=timeout)
        if r.status_code == 200 and int(r.headers.get("Content-Length", "1")) > 0:
            out_path.parent.mkdir(parents=True, exist_ok=True)
            with open(out_path, "wb") as f:
                for chunk in r.iter_content(chunk_size=1<<20):
                    if chunk:
                        f.write(chunk)
            return True, out_path
        else:
            return False, f"HTTP {r.status_code}"
    except Exception as e:
        return False, str(e)

def download_af2_structures(uniprot_id: str, model_field: str | None, out_dir: Path):
    urls = []
    for stem in guess_af2_model_filename(uniprot_id, model_field):
        urls.append(("pdb", f"https://alphafold.ebi.ac.uk/files/{stem}.pdb"))
        urls.append(("cif", f"https://alphafold.ebi.ac.uk/files/{stem}.cif"))
    saved = []
    tried_types = set()
    for fmt, url in urls:
        fname = _safe_filename(url.split("/")[-1])
        if fmt in tried_types:
            # already have one file for this fmt
            continue
        ok, info = try_download(url, out_dir / fname)
        if ok:
            saved.append(out_dir / fname)
            tried_types.add(fmt)
    return saved

def download_rcsb_by_pdb_ids(pdb_ids: list[str], out_dir: Path, max_n: int = 3):
    saved = []
    for i, code in enumerate(pdb_ids[:max_n]):
        code = str(code).strip().upper()[:4]
        if not re.fullmatch(r"[0-9A-Z]{4}", code):
            continue
        url_cif = f"https://files.rcsb.org/download/{code}.cif"
        fname = f"{code}.cif"
        ok, info = try_download(url_cif, out_dir / fname)
        if ok:
            saved.append(out_dir / fname)
        else:
            # Try legacy .pdb as a fallback
            url_pdb = f"https://files.rcsb.org/download/{code}.pdb"
            ok2, info2 = try_download(url_pdb, out_dir / f"{code}.pdb")
            if ok2:
                saved.append(out_dir / f"{code}.pdb")
    return saved

def fetch_structures_for_uniprot(uniprot_id: str, base_dir: Path, max_pdb: int = 3):
    target = base_dir / "downloads" / uniprot_id
    target.mkdir(parents=True, exist_ok=True)

    rows = find_rows_for_uniprot(uniprot_id)
    model_field = None
    pdb_candidates = []
    if rows is not None:
        if "model" in rows.columns:
            model_field = rows.iloc[0]["model"]
        if "PDB" in rows.columns:
            pdb_candidates = [x for x in rows["PDB"].dropna().astype(str).tolist() if x and x != "nan"]

    saved_af2 = download_af2_structures(uniprot_id, model_field, target)
    saved_rcsb = download_rcsb_by_pdb_ids(list(dict.fromkeys(pdb_candidates)), target, max_n=max_pdb)

    links = {
        "alphafold_entry_url": f"https://alphafold.ebi.ac.uk/entry/{uniprot_id}",
        "target_dir": str(target.resolve()),
        "saved_files": [str(p) for p in saved_af2 + saved_rcsb],
        "pdb_ids_tried": pdb_candidates[:max_pdb],
    }
    return links

In [ ]:
#@title Demo: Download structures for a UniProt ID
UNIPROT_ID = "A0A1D8PJF9"  #@param {type:"string"}
MAX_PDB = 3                 #@param {type:"integer"}

links = fetch_structures_for_uniprot(UNIPROT_ID, BASE_DIR, max_pdb=MAX_PDB)
print("AlphaFold entry:", links["alphafold_entry_url"])
print("Saved to folder:", links["target_dir"])
print("Files saved:")
for p in links["saved_files"]:
    print("  -", p)
if links["pdb_ids_tried"]:
    print("PDB IDs attempted:", links["pdb_ids_tried"])
else:
    print("No PDB IDs found in PDBvsAF2-Table1 for this UniProt (experimental download may be empty).")

AlphaFold entry: https://alphafold.ebi.ac.uk/entry/A0A1D8PJF9
Saved to folder: /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/downloads/A0A1D8PJF9
Files saved:
  - /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/downloads/A0A1D8PJF9/AF-A0A1D8PJF9-F1-model_v4.pdb
  - /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/downloads/A0A1D8PJF9/AF-A0A1D8PJF9-F1-model_v4.cif
  - /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/downloads/A0A1D8PJF9/6DEO.cif
PDB IDs attempted: ['6DEO']


In [ ]:
#@title Quick Count sequences & captions (SwissProtCLAP)
from pathlib import Path
import json, csv, sys
import pandas as pd
import lmdb, pickle

BASE_DIR = Path(BASE_DIR)  # uses your existing BASE_DIR
SP_DIR = BASE_DIR / "SwissProtCLAP"
STAB_DIR = BASE_DIR / "downstream_Editing" / "datasets_and_checkpoints" / "stability"
REGION_DIR = BASE_DIR / "downstream_Editing" / "datasets_and_checkpoints" / "region"
MISATO_DIR = BASE_DIR / "downstream_Editing" / "datasets_and_checkpoints" / "peptide_binding" / "MISATO"
STRUCT_DIR = BASE_DIR / "downstream_Editing" / "datasets_and_checkpoints" / "structure"

seq_path = SP_DIR / "protein_sequence.txt"
txt_path = SP_DIR / "text_sequence.txt"

def count_lines(path: Path):
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        return sum(1 for _ in f)

def two_line_pairs_iter(path: Path):
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        while True:
            id_line = f.readline()
            if not id_line:
                break
            val_line = f.readline()
            if not val_line:
                break
            yield id_line.strip(), val_line.strip()

def verify_swissprotclap(seq_path: Path, txt_path: Path):
    assert seq_path.exists(), f"Missing: {seq_path}"
    assert txt_path.exists(), f"Missing: {txt_path}"

    n_seq_lines = count_lines(seq_path)
    n_txt_lines = count_lines(txt_path)
    n_seq_pairs = n_seq_lines // 2
    n_txt_pairs = n_txt_lines // 2

    # Check alignment and duplicates
    dup_ids = set()
    seen_ids = set()
    mismatches = 0
    empty_seq = 0
    empty_cap = 0
    total_pairs = 0
    for (sid, seq), (tid, cap) in zip(two_line_pairs_iter(seq_path), two_line_pairs_iter(txt_path)):
        total_pairs += 1
        if sid != tid:
            mismatches += 1
        if not seq:
            empty_seq += 1
        if not cap:
            empty_cap += 1
        if sid in seen_ids:
            dup_ids.add(sid)
        else:
            seen_ids.add(sid)

    return {
        "seq_lines": n_seq_lines,
        "txt_lines": n_txt_lines,
        "seq_records": n_seq_pairs,
        "txt_records": n_txt_pairs,
        "paired_records_checked": total_pairs,
        "id_mismatches": mismatches,
        "empty_sequences": empty_seq,
        "empty_captions": empty_cap,
        "unique_ids": len(seen_ids),
        "duplicate_id_count": len(dup_ids),
    }

def count_stability(stab_dir: Path):
    out = {}
    for jf in ["stability_train.json", "stability_valid.json", "stability_test.json"]:
        p = stab_dir / jf
        if p.exists():
            data = json.loads(p.read_text())
            # count how many have an obvious sequence field
            seq_keys = ["sequence", "protein", "mutated_seq", "seq"]
            n_with_seq = 0
            for d in data:
                if any(isinstance(d.get(k), str) and d.get(k) for k in seq_keys):
                    n_with_seq += 1
            out[jf] = {"entries": len(data), "with_sequence": n_with_seq}
        else:
            out[jf] = {"entries": 0, "with_sequence": 0, "missing": True}
    return out

def count_region(region_dir: Path):
    out = {}
    p_raw = region_dir / "raw_sequence.csv"
    p_pre = region_dir / "preprocessed_data.csv"
    p_pdb = region_dir / "PDBvsAF2-Table1.csv"
    p_tdark = region_dir / "TdarkAF2-Table1.csv"

    if p_raw.exists():
        df = pd.read_csv(p_raw)
        out["raw_sequence.csv"] = {"rows": len(df), "columns": list(df.columns)}
    if p_pre.exists():
        try:
            df = pd.read_csv(p_pre)
        except Exception:
            df = pd.read_csv(p_pre, header=None)
            df.columns = ["AA"]
        out["preprocessed_data.csv"] = {"rows": len(df), "columns": list(df.columns)}
    if p_pdb.exists():
        df = pd.read_csv(p_pdb)
        uni_cols = [c for c in df.columns if c.lower().startswith("uniprot")]
        pdb_cols = [c for c in df.columns if c.upper()=="PDB"]
        n_uni = df[uni_cols[0]].nunique() if uni_cols else None
        n_pdb = df[pdb_cols[0]].nunique() if pdb_cols else None
        out["PDBvsAF2-Table1.csv"] = {
            "rows": len(df),
            "unique_uniprot": n_uni,
            "unique_pdb_ids": n_pdb,
            "columns": list(df.columns)
        }
    if p_tdark.exists():
        df = pd.read_csv(p_tdark)
        uni_cols = [c for c in df.columns if c.lower().startswith("uniprot")]
        n_uni = df[uni_cols[0]].nunique() if uni_cols else None
        out["TdarkAF2-Table1.csv"] = {
            "rows": len(df),
            "unique_uniprot": n_uni,
            "columns": list(df.columns)
        }
    return out

def count_misato(misato_dir: Path):
    out = {}
    p_json = misato_dir / "PDB_idx2text.json"
    p_map = misato_dir / "PDB_mapping_data.txt"
    p_pre = misato_dir / "preprocessed_data.csv"
    if p_json.exists():
        js = json.loads(p_json.read_text())
        out["PDB_idx2text.json"] = {"entries": len(js)}
    if p_map.exists():
        with open(p_map, "r") as f:
            n = sum(1 for _ in f if _.strip())
        out["PDB_mapping_data.txt"] = {"non_empty_lines": n}
    if p_pre.exists():
        try:
            df = pd.read_csv(p_pre)
        except Exception:
            df = pd.read_csv(p_pre, header=None)
        out["preprocessed_data.csv"] = {"rows": len(df), "columns": list(df.columns)}
    return out

def count_structure_lmdbs(struct_dir: Path):
    out = {}
    lmdb_names = [
        "secondary_structure_train.lmdb",
        "secondary_structure_valid.lmdb",
        "secondary_structure_ts115.lmdb",
        "secondary_structure_casp12.lmdb",
        "secondary_structure_cb513.lmdb",
    ]
    for name in lmdb_names:
        p = struct_dir / name
        if not p.exists():
            out[name] = {"entries": 0, "missing": True}
            continue
        env = lmdb.open(str(p), readonly=True, lock=False, readahead=False, max_readers=1024)
        n = 0
        with env.begin(write=False) as txn:
            c = txn.cursor()
            for _ in c:
                n += 1
        env.close()
        out[name] = {"entries": n}
    return out

### ---- Run all counts ----
swiss = verify_swissprotclap(seq_path, txt_path)
stab = count_stability(STAB_DIR)
region = count_region(REGION_DIR)
misato = count_misato(MISATO_DIR)
struct = count_structure_lmdbs(STRUCT_DIR)

print("=== SwissProtCLAP (main caption ↔ sequence pairs) ===")
for k, v in swiss.items():
    print(f"{k:>24s}: {v}")

print("\n=== Stability (JSON) ===")
print(pd.DataFrame(stab).T)

print("\n=== Region ===")
print(pd.DataFrame(region).T)

print("\n=== MISATO (peptide binding) ===")
print(pd.DataFrame(misato).T)

print("\n=== Structure LMDBs ===")
print(pd.DataFrame(struct).T)

# Final headline numbers for your experiment planning:
print("\n--- Headline ---")
print(f"SwissProtCLAP pairs (caption+sequence): {swiss['paired_records_checked']} (seq_records file says {swiss['seq_records']}, txt_records file says {swiss['txt_records']})")
if swiss["id_mismatches"] == 0 and swiss["seq_records"] == swiss["txt_records"]:
    print("SwissProtCLAP files are line-aligned with identical record counts.")
else:
    print("SwissProtCLAP files show mismatches or unequal record counts — inspect above.")

=== SwissProtCLAP (main caption ↔ sequence pairs) ===
               seq_lines: 1082318
               txt_lines: 1082318
             seq_records: 541159
             txt_records: 541159
  paired_records_checked: 541159
           id_mismatches: 0
         empty_sequences: 0
          empty_captions: 0
              unique_ids: 541159
      duplicate_id_count: 0

=== Stability (JSON) ===
                      entries  with_sequence
stability_train.json    53614              0
stability_valid.json     2512              0
stability_test.json     12851              0

=== Region ===
                       rows  \
raw_sequence.csv       1810   
preprocessed_data.csv    58   
PDBvsAF2-Table1.csv    1810   
TdarkAF2-Table1.csv    5991   

                                                                 columns  \
raw_sequence.csv                                           [uniprot, AA]   
preprocessed_data.csv  [MKLLTSLVFCSLLLGVCHGGFFSFIGEAFQGAGDMWRAYTDMKEA...   
PDBvsAF2-Table1.csv    [PDB,

In [ ]:
#@title Build a 1k subset manifest (IDs, lengths, caption preview, PDB IDs)
from pathlib import Path
import pandas as pd
import random

# Uses your existing BASE_DIR (can be a Drive folder)
BASE_DIR = Path(BASE_DIR)  # do not change if already set above
SP_DIR = BASE_DIR / "SwissProtCLAP"
REGION_DIR = BASE_DIR / "downstream_Editing" / "datasets_and_checkpoints" / "region"
OUT_DIR = BASE_DIR / "subset_5k"
OUT_DIR.mkdir(parents=True, exist_ok=True)

SEQ_PATH = SP_DIR / "protein_sequence.txt"
TXT_PATH = SP_DIR / "text_sequence.txt"
PDB_AF2_PATH = REGION_DIR / "PDBvsAF2-Table1.csv"

SUBSET_SIZE = 5000  # adjust if you want
RANDOM_SEED = 42    # to make selection reproducible
CAPTION_PREVIEW_CHARS = 220

def pair_iter(path: Path):
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        while True:
            id_line = f.readline()
            if not id_line:
                break
            val_line = f.readline()
            if not val_line:
                break
            yield id_line.strip(), val_line.strip()

# 1) Load all IDs + lengths from protein_sequence.txt
ids, lens = [], []
for pid, seq in pair_iter(SEQ_PATH):
    ids.append(pid)
    lens.append(len(seq))

# Sample 1k
random.seed(RANDOM_SEED)
idxs = random.sample(range(len(ids)), SUBSET_SIZE)
subset_ids = [ids[i] for i in idxs]
subset_lens = [lens[i] for i in idxs]
subset_set = set(subset_ids)

# 2) Build caption previews for the chosen set
id_to_caption = {}
for pid, cap in pair_iter(TXT_PATH):
    if pid in subset_set:
        id_to_caption[pid] = cap[:CAPTION_PREVIEW_CHARS]

# 3) Map to (optional) experimental PDB IDs via PDBvsAF2 table (first PDB per UniProt)
id_to_pdb = {}
if PDB_AF2_PATH.exists():
    df = pd.read_csv(PDB_AF2_PATH)
    # use flexible UniProt column name
    uni_col = [c for c in df.columns if c.lower().startswith("uniprot")]
    if uni_col:
        uni_col = uni_col[0]
        for uid, sub in df[df[uni_col].astype(str).isin(subset_set)].groupby(uni_col):
            # Collect all PDB IDs in this group (there can be multiple rows)
            pdbs = [p for p in sub["PDB"].astype(str).tolist() if p and p != "nan"]
            id_to_pdb[uid] = pdbs[0] if pdbs else None

# 4) Write manifest CSV
rows = []
for pid, L in zip(subset_ids, subset_lens):
    rows.append({
        "uniprot_id": pid,
        "length": L,
        "caption_preview": id_to_caption.get(pid, None),
        "pdb_id": id_to_pdb.get(pid, None)
    })
manifest = pd.DataFrame(rows)
manifest_path = OUT_DIR / "subset_manifest_5k.csv"
manifest.to_csv(manifest_path, index=False)
print(f"Saved: {manifest_path}  shape={manifest.shape}")
manifest.head()

Saved: /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/subset_5k/subset_manifest_5k.csv  shape=(5000, 4)


,uniprot_id,length,caption_preview,pdb_id
0,Q45744,649,Promotes colloidosmotic lysis by binding to th...,None
1,P59712,448,Packages the positive strand viral genome RNA ...,None
2,A9N7Y7,72,One of the essential components for the initia...,None
3,O52759,206,"One of the primary rRNA binding proteins, it b...",None
4,Q9YGT2,306,Sequence-specific transcription factor which i...,None


In [ ]:
#@title  Download AF2 v2 PDB + RCSB experimental mmCIF for the subset
import re, time, requests, pandas as pd, json
from pathlib import Path
from tqdm import tqdm

BASE_DIR = Path(BASE_DIR)
OUT_DIR = BASE_DIR / "subset_1k"
DL_DIR = OUT_DIR / "downloads"
DL_DIR.mkdir(parents=True, exist_ok=True)

manifest_path = OUT_DIR / "subset_manifest_1k.csv"
df = pd.read_csv(manifest_path)

# Toggles
ALLOW_FALLBACK_V4 = False   # strictly v2 by default; set True to try v4 if v2 missing
MAX_RCSB_PER_PROTEIN = 1    # download at most one experimental structure per UniProt
SLEEP_BETWEEN = 0.05        # small pause to be polite

def safe_name(s: str) -> str:
    return re.sub(r"[^0-9A-Za-z._-]+", "_", s)

def af2_urls_v2(uniprot_id: str):
    stem = f"AF-{uniprot_id}-F1-model_v2"
    return [f"https://alphafold.ebi.ac.uk/files/{stem}.pdb"]

def af2_urls_v4(uniprot_id: str):
    stem = f"AF-{uniprot_id}-F1-model_v4"
    return [f"https://alphafold.ebi.ac.uk/files/{stem}.pdb"]

def rcsb_urls(pdb_id: str):
    pdb_id = pdb_id.strip().upper()[:4]
    return [f"https://files.rcsb.org/download/{pdb_id}.cif"]

def try_download(url: str, out_path: Path, timeout=60):
    try:
        r = requests.get(url, stream=True, timeout=timeout, allow_redirects=True)
        if r.status_code == 200 and int(r.headers.get("Content-Length", "1")) > 0:
            out_path.parent.mkdir(parents=True, exist_ok=True)
            with open(out_path, "wb") as f:
                for chunk in r.iter_content(chunk_size=1<<20):
                    if chunk:
                        f.write(chunk)
            return True, out_path
        return False, f"HTTP {r.status_code}"
    except Exception as e:
        return False, str(e)

records = []
total_bytes = 0

for _, row in tqdm(df.iterrows(), total=len(df)):
    uid = row["uniprot_id"]
    pdb_hint = str(row["pdb_id"]) if "pdb_id" in row and pd.notna(row["pdb_id"]) else None

    target = DL_DIR / uid
    target.mkdir(parents=True, exist_ok=True)

    saved = []
    # 1) AF2 v2 PDB
    for url in af2_urls_v2(uid):
        ok, info = try_download(url, target / safe_name(url.split("/")[-1]))
        if ok:
            saved.append(str(info))
            total_bytes += Path(info).stat().st_size
            break
        elif ALLOW_FALLBACK_V4:
            for url2 in af2_urls_v4(uid):
                ok2, info2 = try_download(url2, target / safe_name(url2.split("/")[-1]))
                if ok2:
                    saved.append(str(info2))
                    total_bytes += Path(info2).stat().st_size
                    break
        time.sleep(SLEEP_BETWEEN)

    # 2) Experimental mmCIF (RCSB) — if PDB id present
    if pdb_hint:
        tried = 0
        for url in rcsb_urls(pdb_hint):
            ok, info = try_download(url, target / safe_name(url.split("/")[-1]))
            if ok:
                saved.append(str(info))
                total_bytes += Path(info).stat().st_size
                tried += 1
            time.sleep(SLEEP_BETWEEN)
            if tried >= MAX_RCSB_PER_PROTEIN:
                break

    records.append({
        "uniprot_id": uid,
        "saved_files": json.dumps(saved, ensure_ascii=False)
    })

# Write a download manifest
dl_manifest = pd.DataFrame(records)
dl_manifest_path = OUT_DIR / "subset_downloads_manifest.csv"
dl_manifest.to_csv(dl_manifest_path, index=False)

# Quick size summary
def human(n):
    for unit in ["B","KB","MB","GB","TB"]:
        if n < 1024: return f"{n:.2f} {unit}"
        n /= 1024
    return f"{n:.2f} PB"

print(f"\nSaved download manifest: {dl_manifest_path}")
print(f"Total size downloaded (approx): {human(total_bytes)}")
print(f"Files saved under: {DL_DIR}")

100%|██████████| 1000/1000 [06:45<00:00,  2.46it/s]


Saved download manifest: /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/subset_1k/subset_downloads_manifest.csv
Total size downloaded (approx): 492.18 MB
Files saved under: /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/subset_1k/downloads


In [ ]:
#@title Example: read sequence, caption, and a structure file (PDB/mmCIF) for a UniProt ID
# Uses your existing BASE_DIR and (optionally) your subset_1k manifests/downloads
from pathlib import Path
import pandas as pd
import re

# Install biopython if needed for parsing structures
try:
    import Bio
except Exception:
    !pip -q install biopython
from Bio.PDB.MMCIFParser import MMCIFParser
from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.Polypeptide import PPBuilder

# Inputs
UNIPROT_ID = "P50405"  #@param {type:"string"}  # pick one from your subset (this example has 6VYN in your printout)
PREFER_EXPERIMENTAL = True  #@param {type:"boolean"}  # if True, pick RCSB mmCIF first (if downloaded); else prefer AF2 PDB

# Paths (assumes you've already set BASE_DIR earlier in the notebook)
BASE_DIR = Path(BASE_DIR)
SP_DIR = BASE_DIR / "SwissProtCLAP"
SUBSET_DIR = BASE_DIR / "subset_1k"
DL_DIR = SUBSET_DIR / "downloads" / UNIPROT_ID

SEQ_PATH = SP_DIR / "protein_sequence.txt"
TXT_PATH = SP_DIR / "text_sequence.txt"

# Optional manifests:
MANIFESTS = [
    SUBSET_DIR / "subset_manifest_1k_rcsb.csv",  # after RCSB augmentation
    SUBSET_DIR / "subset_manifest_1k.csv",       # original 1k manifest
]

def _pair_iter(path: Path):
    """Yield (id, value) pairs from a two-line-per-record text file."""
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        while True:
            id_line = f.readline()
            if not id_line:
                break
            val_line = f.readline()
            if not val_line:
                break
            yield id_line.strip(), val_line.strip()

def read_seq_and_caption(uniprot_id: str):
    """Return (sequence, caption) for UniProt ID from SwissProtCLAP."""
    seq = cap = None
    # scan sequence file
    for pid, s in _pair_iter(SEQ_PATH):
        if pid == uniprot_id:
            seq = s
            break
    # scan caption file
    for pid, t in _pair_iter(TXT_PATH):
        if pid == uniprot_id:
            cap = t
            break
    return seq, cap

def find_pdb_id_in_manifests(uniprot_id: str):
    """Look up PDB ID from your subset manifest(s). Prefer 'pdb_id_rcsb' if present, else 'pdb_id'."""
    for man in MANIFESTS:
        if not man.exists():
            continue
        df = pd.read_csv(man)
        row = df[df["uniprot_id"].astype(str) == uniprot_id]
        if row.empty:
            continue
        # prefer RCSB column if it exists
        for col in ["pdb_id_rcsb", "pdb_id"]:
            if col in row.columns:
                val = row.iloc[0][col]
                if isinstance(val, str) and val.strip() and val.strip().lower() != "nan":
                    return val.strip().upper()[:4]
    return None

def choose_structure_file(uniprot_id: str, prefer_experimental: bool = True):
    """
    Choose a local structure file path to show:
      - If prefer_experimental: look for mmCIF/PDB under subset_1k/downloads/<ID>/ that are NOT AF2 files.
      - Otherwise: prefer AF2 PDB (AF-<ID>-F1-model_v*.pdb) if present.
    Returns Path or None.
    """
    d = SUBSET_DIR / "downloads" / uniprot_id
    if not d.exists():
        return None
    af2_stem = f"AF-{uniprot_id}-"
    exp_first = sorted([p for p in d.glob("*.cif") if af2_stem not in p.name]) + \
                sorted([p for p in d.glob("*.pdb") if af2_stem not in p.name])
    af2_first = sorted(d.glob(f"AF-{uniprot_id}-F1-model_v*.pdb")) + sorted(d.glob(f"AF-{uniprot_id}-F1-model_v*.cif"))
    if prefer_experimental and exp_first:
        return exp_first[0]
    if af2_first:
        return af2_first[0]
    if exp_first:
        return exp_first[0]
    return None

def parse_structure_basic(path: Path):
    """
    Parse a PDB or mmCIF file and return a small summary dict:
      chains, residues (approx), first polypeptide sequence (if parseable).
    """
    path = Path(path)
    if path.suffix.lower() == ".cif":
        parser = MMCIFParser(QUIET=True)
    else:
        parser = PDBParser(QUIET=True)

    structure = parser.get_structure("model", str(path))
    model = next(structure.get_models())  # first model
    chains = list(model.get_chains())

    # Approx residue count
    res_count = sum(1 for _ in model.get_residues())

    # Try extracting an AA sequence from coordinates (may be partial/missing for some entries)
    ppb = PPBuilder()
    polypeptides = ppb.build_peptides(model)
    seq_example = str(polypeptides[0].get_sequence()) if polypeptides else None

    return {
        "file": str(path),
        "n_chains": len(chains),
        "approx_residues": res_count,
        "sequence_from_structure_first_chain": seq_example[:100] + "..." if seq_example and len(seq_example) > 100 else seq_example
    }

# ---- Run the example ----
seq, cap = read_seq_and_caption(UNIPROT_ID)
pdb_code = find_pdb_id_in_manifests(UNIPROT_ID)
struct_path = choose_structure_file(UNIPROT_ID, prefer_experimental=PREFER_EXPERIMENTAL)

print("UniProt ID: ", UNIPROT_ID)
print("Sequence:   ", (seq[:100] + "..." if seq and len(seq) > 100 else seq))
print("Seq length: ", len(seq) if seq else None)
print("Caption:    ", (cap[:200] + "..." if cap and len(cap) > 200 else cap))
print("PDB code (from manifest):", pdb_code)
print("Chosen structure file:   ", struct_path)

if struct_path and Path(struct_path).exists():
    info = parse_structure_basic(struct_path)
    print("\n[Structure summary]")
    for k, v in info.items():
        print(f" - {k}: {v}")
else:
    print("\n[Structure summary]")
    print(" - No local structure file found for this ID under subset_1k/downloads/.")
    if pdb_code:
        print("   You have a PDB code; run the earlier download cell to fetch the mmCIF.")
    else:
        print("   No PDB code in manifest; use AF2 download (or run the RCSB augmentation cell to search by UniProt).")

UniProt ID:  P50405
Sequence:    MAKSHLLQWLLLLPTLCCPGAAITSASSLECAQGPQFWCQSLEHAVQCRALGHCLQEVWGHAGANDLCQECEDIVHLLTKMTKEDAFQEAIRKFLEQECD...
Seq length:  377
Caption:     Pulmonary surfactant-associated proteins promote alveolar stability by lowering the surface tension at the air-liquid interface in the peripheral air spaces. SP-B increases the collapse pressure of pa...
PDB code (from manifest): 6VYN
Chosen structure file:    /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/subset_1k/downloads/P50405/6VYN.cif

[Structure summary]
 - file: /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/subset_1k/downloads/P50405/6VYN.cif
 - n_chains: 8
 - approx_residues: 827
 - sequence_from_structure_first_chain: DLCQECEDIVHLLTKMTKEDAFQEAIRKFLEQECDILPLKLLVPRCRQVLDVYLPLVIDYFQSQINPKAICNHVGLCPR


### Generate CoT without ground truth

In [ ]:
!pip install openai
!pip install anthropic
import asyncio
import json, os, re, tempfile, time, math
from datetime import datetime
from typing import List, Dict, Any, Iterable
from openai import OpenAI
import torch
import openai
import abc
from transformers import AutoTokenizer, AutoModelForCausalLM
import math
from concurrent.futures import ThreadPoolExecutor
from pathlib import Path
from tqdm import tqdm
import anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 9.0 MB/s eta 0:00:00


In [ ]:
# Set the required environment variables.
#os.environ["ANTHROPIC_API_KEY"] =
#os.environ["MISTRAL_API_KEY"] =
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
#anthropic_client = anthropic.Anthropic(api_key=os.environ["ANTHROPIC_API_KEY"])
OPENAI_POLL_INTERVAL = 5
ANTHROPIC_POLL_INTERVAL = 5
OPENAI_CHUNK_SIZE = 300
ANTHROPIC_MAX_BATCH = 300

In [ ]:

# ===================== GPT Batch: Protein → Description (CoT) with In-Prompt Example =====================
# 1) CONFIG
import os, json, re, html, random
from pathlib import Path
from typing import Dict, Any, List, Tuple
from openai import OpenAI

# --- Required ---
os.environ.setdefault("OPENAI_API_KEY", "<PUT_YOUR_KEY_HERE>")  # or set in env before running
BASE_DIR         = Path("")  # ProteinDT root
MODEL            = "gpt-4o"        # e.g., "gpt-4o", "o1-mini", "o1-preview"
N_SAMPLES        = 100            # how many proteins to generate
TEMPERATURE      = 0.1
MAX_TOKENS       = 4000
BATCH_COMPLETION_WINDOW = "24h"
RANDOM_SEED      = 43

# --- Optional: restrict to your 1k manifest (reuse your subset) ---
MANIFEST_PATH    = BASE_DIR / "subset_1k" / "subset_manifest_1k.csv"  # or set to None to disable
MANIFEST_LIMIT   = 0    # 0 = no extra limit

# --- Output files ---
OUT_DIR            = BASE_DIR / "gpt_batch_protein2desc"
OUT_DIR.mkdir(parents=True, exist_ok=True)
BATCH_INPUT_JSONL  = OUT_DIR / "protein2desc_input.jsonl"
BATCH_META_JSON    = OUT_DIR / "protein2desc_meta.json"     # custom_id → {uniprot_id, seq, ground_truth}
BATCH_INFO_JSON    = OUT_DIR / "protein2desc_batch_info.json"
BATCH_OUTPUT_JSONL = OUT_DIR / "protein2desc_output.jsonl"  # downloaded later
SFT_OUT_JSON       = OUT_DIR / "protein2desc_cot_sft.json"  # built later

# ===================== 2) Helpers & data =====================
client = OpenAI()

def strip_html(s: str) -> str:
    if not isinstance(s, str): return ""
    s = html.unescape(s)
    return re.sub(r"<[^>]+>", " ", s).replace("\xa0"," ").strip()

def trunc(s: str, n: int) -> str:
    s = s or ""
    return (s[:n] + "…") if len(s) > n else s

def two_line_pairs(path: Path):
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        while True:
            id_line = f.readline()
            if not id_line: break
            val_line = f.readline()
            if not val_line: break
            yield id_line.strip(), val_line.strip()

def load_swissprotclap(base_dir: Path):
    sp = base_dir / "SwissProtCLAP"
    seq_fp, txt_fp = sp / "protein_sequence.txt", sp / "text_sequence.txt"
    assert seq_fp.exists() and txt_fp.exists(), f"Missing SwissProtCLAP under {sp}"
    id2seq = {pid: seq for pid, seq in two_line_pairs(seq_fp)}
    id2cap = {pid: strip_html(txt) for pid, txt in two_line_pairs(txt_fp)}
    ids = [i for i in id2seq.keys() if i in id2cap]
    return ids, id2seq, id2cap

# ===================== 3) Build SYSTEM prompt with a real example =====================
random.seed(RANDOM_SEED)
ids, id2seq, id2cap = load_swissprotclap(BASE_DIR)

# Optional restriction to your manifest subset
restrict_ids = None
if MANIFEST_PATH and MANIFEST_PATH.exists():
    import pandas as pd
    dfm = pd.read_csv(MANIFEST_PATH)
    restrict_ids = [str(x) for x in dfm["uniprot_id"].astype(str).tolist()]
    if MANIFEST_LIMIT and MANIFEST_LIMIT > 0:
        restrict_ids = restrict_ids[:MANIFEST_LIMIT]

pool = restrict_ids if restrict_ids else ids[:]
random.shuffle(pool)

# --- Pick ONE random example from the dataset for the in-prompt "Example:" ---
ex_uid = random.choice(pool)
ex_seq = id2seq[ex_uid]
ex_gt  = id2cap[ex_uid]

PROTEIN_PROMPT_WITH_EXAMPLE = (
    "You are a professional protein biologist. Your task is to generate a natural, concise, and biologically "
    "accurate description of a protein based **only** on its amino-acid sequence.\n\n"
    "### Example:\n"
    f"Protein UniProt: {ex_uid}\n"
    f"Protein sequence: {trunc(ex_seq, 1200)}\n"
    f"Answer: <answer> {ex_gt} </answer>\n\n"
    "### Now try this:\n"
    "Given ONLY the sequence, first think step by step about plausible features (signal peptides, transmembrane "
    "helices, repeats, low-complexity segments, catalytic motifs/domains, localization signals), then produce a "
    "polished 2–4 sentence description.\n\n"
    "Your final answer **must** be returned in the format:\n"
    "<thinking>\n[3 - 5 steps: Your reasoning steps using sequence-derived evidence]\n</thinking>\n\n"
    "<answer>\n[2–4 sentences; A natural, concise scientific description of the protein]\n</answer>\n"
    #"Do not include database IDs, citations, or organism names unless they follow directly from sequence class."
)

def build_messages_for_generation(uniprot_id: str, seq: str) -> list[dict]:
    # System includes the example; user supplies the actual sequence to describe
    user = (
        f"Protein UniProt: {uniprot_id}\n"
        f"Protein sequence (truncated to 2000 aa if long):\n{trunc(seq, 2000)}\n\n"
        "Think step by step in <thinking>, then provide the final textual description in <answer>."
    )
    return [
        {"role": "system", "content": PROTEIN_PROMPT_WITH_EXAMPLE},
        {"role": "user", "content": user},
    ]

# ===================== 4) Build JSONL (exclude example from targets), submit batch =====================
# Exclude the example ID from the generation set to avoid leakage
usable_pool = [x for x in pool if x != ex_uid]
chosen = usable_pool[:N_SAMPLES]

meta = {}
with open(BATCH_INPUT_JSONL, "w", encoding="utf-8") as fout:
    for i, uid in enumerate(chosen):
        seq = id2seq[uid]
        gt  = id2cap[uid]  # ground truth caption (NOT shown to model)
        custom_id = f"p2d_{i}_{uid}"
        meta[custom_id] = {
            "uniprot_id": uid,
            "sequence": trunc(seq, 2000),
            "ground_truth": gt,
            "example_uid": ex_uid  # for bookkeeping
        }
        body = {
            "model": MODEL,
            "temperature": TEMPERATURE,
            "messages": build_messages_for_generation(uid, seq),
            "max_tokens": MAX_TOKENS,
        }
        line = {
            "custom_id": custom_id,
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": body,
        }
        fout.write(json.dumps(line, ensure_ascii=False) + "\n")

BATCH_META_JSON.write_text(json.dumps(meta, ensure_ascii=False, indent=2), encoding="utf-8")
print(f"[Example used in prompt] UniProt: {ex_uid}")
print(f"Wrote batch input: {BATCH_INPUT_JSONL}")
print(f"Wrote meta map:   {BATCH_META_JSON}")

# Upload and create batch
with open(BATCH_INPUT_JSONL, "rb") as f:
    input_file = client.files.create(file=f, purpose="batch")

batch = client.batches.create(
    input_file_id=input_file.id,
    endpoint="/v1/chat/completions",
    completion_window=BATCH_COMPLETION_WINDOW,
)

BATCH_INFO_JSON.write_text(json.dumps({
    "batch_id": batch.id,
    "input_file_id": input_file.id,
    "status": batch.status,
    "example_uid": ex_uid
}, indent=2), encoding="utf-8")

print("\n=== SUBMITTED BATCH ===")
print("batch_id:      ", batch.id)
print("input_file_id: ", input_file.id)
print("status:        ", batch.status)
print(f"(Saved to {BATCH_INFO_JSON})")

print("\nFinish later with your existing fetch/assemble cell (no changes needed).")

[Example used in prompt] UniProt: A4FZG7
Wrote batch input: /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc/protein2desc_input.jsonl
Wrote meta map:   /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc/protein2desc_meta.json

=== SUBMITTED BATCH ===
batch_id:       batch_68d5b50cae4881909cebe6c9b9354a10
input_file_id:  file-JBbXzdvBHfMyDtuFjvijye
status:         validating
(Saved to /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc/protein2desc_batch_info.json)

Finish later with your existing fetch/assemble cell (no changes needed).


### Go gpt dashboard to check whether the batch finished processing, once done.

[Example used in prompt] UniProt: A4FZG7
Wrote batch input: /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc/protein2desc_input.jsonl
Wrote meta map:   /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc/protein2desc_meta.json

=== SUBMITTED BATCH ===
batch_id:       batch_68d5b35e7e348190b08435bcb0778017
input_file_id:  file-2yJezBfdPgJ9id1vt9CwK8
status:         validating
(Saved to /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc/protein2desc_batch_info.json)

Finish later with your existing fetch/assemble cell (no changes needed).


In [ ]:
# ===================== Fetch output & build SFTs (parsed + raw) =====================
from pathlib import Path
import json, re
from openai import OpenAI

# Your project root (folder that contains SwissProtCLAP/, etc.)
#BASE_DIR = Path("/content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT")  # <-- change if needed

# All artifacts now live under BASE_DIR / "gpt_batch_protein2desc"
OUT_DIR            = BASE_DIR / "gpt_batch_protein2desc"
BATCH_INFO_JSON    = OUT_DIR / "protein2desc_batch_info.json"
BATCH_OUTPUT_JSONL = OUT_DIR / "protein2desc_output.jsonl"
BATCH_META_JSON    = OUT_DIR / "protein2desc_meta.json"

# Outputs (two variants)
SFT_OUT_JSON_PARSED = OUT_DIR / "protein2desc_cot_sft.json"   # parsed <thinking>/<answer>
SFT_OUT_JSON_RAW    = OUT_DIR / "protein2desc_cot_raw.json"   # raw assistant text, no parsing

client = OpenAI()

client = OpenAI()

def fetch_batch_output_if_ready(batch_id: str, out_jsonl: Path):
    """If batch is complete, download output JSONL; else return current status."""
    b = client.batches.retrieve(batch_id)
    if not getattr(b, "output_file_id", None):
        return False, str(b.status)
    content = client.files.content(b.output_file_id)
    data = content.read()
    out_jsonl.write_bytes(data)
    return True, out_jsonl

def extract_assistant_content(obj) -> str:
    """Robust extractor for Batch lines: response.body.choices[0].message.content."""
    resp = obj.get("response") or {}
    body = resp.get("body") if isinstance(resp, dict) else None
    src  = body if isinstance(body, dict) else resp
    try:
        return src["choices"][0]["message"]["content"]
    except Exception:
        return ""

def extract_request_messages(obj):
    """Return (system_content, user_content) from the *request* body messages for this line."""
    req = obj.get("request") or {}
    rbody = req.get("body") if isinstance(req, dict) else None
    src = rbody if isinstance(rbody, dict) else req
    msgs = src.get("messages", []) if isinstance(src, dict) else []
    sys_txt, usr_txt = None, None
    for m in msgs:
        if sys_txt is None and m.get("role") == "system":
            sys_txt = m.get("content", "")
    for m in reversed(msgs):
        if m.get("role") == "user":
            usr_txt = m.get("content", "")
            break
    return sys_txt or "", usr_txt or ""

# -------- Loose tag parser (handles missing closing tags) --------
def _extract_tag_loose(text: str, tag: str):
    """Return (content, endpos, found) for <tag> ... [</tag>] allowing missing closing tag."""
    if not text:
        return "", 0, False
    open_pat  = re.compile(fr"<{tag}\b[^>]*>", re.I)
    close_pat = re.compile(fr"</{tag}\s*>", re.I)
    m_open = open_pat.search(text)
    if not m_open:
        return "", 0, False
    m_close = close_pat.search(text, m_open.end())
    if m_close:
        content = text[m_open.end():m_close.start()]
        endpos  = m_close.end()
    else:
        # no closing tag → capture to end
        content = text[m_open.end():]
        endpos  = len(text)
    # clean up stray trailing characters often seen in CSV dumps: quotes/commas
    content = content.strip().rstrip('", \t\r\n')
    return content, endpos, True

def parse_thinking_answer_loose(text: str):
    """
    Extract thinking and answer with tolerant rules:
     - If <thinking> found and closed: use between tags.
     - If <thinking> open only: capture until <answer> (if exists) or end.
     - If <answer> found and closed: use between tags.
     - If <answer> open only: capture to end.
     - If neither found: thinking=text, answer=""
    """
    if not text:
        return "", ""
    # thinking first
    thinking, th_end, th_found = _extract_tag_loose(text, "thinking")
    remainder = text
    if th_found:
        remainder = text[th_end:]
    # then answer from remainder or full text
    answer, _, an_found = _extract_tag_loose(remainder, "answer")
    # fallback: if no thinking tag, but answer exists, set thinking to text up to start of answer
    if not th_found and an_found:
        # re-find the open answer to split
        m_open_ans = re.search(r"<answer\b[^>]*>", remainder, re.I)
        cutoff = m_open_ans.start() if m_open_ans else len(text)
        pre = text[:cutoff].strip().rstrip('", \t\r\n')
        thinking = pre if pre else thinking
    # final fallbacks
    if not th_found and not an_found:
        thinking = text.strip().rstrip('", \t\r\n')
        answer   = ""
    return thinking, answer

# ---------------------------- main ----------------------------
assert BATCH_INFO_JSON.exists(), f"Missing {BATCH_INFO_JSON}"
assert BATCH_META_JSON.exists(), f"Missing {BATCH_META_JSON}"

binfo = json.loads(BATCH_INFO_JSON.read_text())
batch_id = binfo["batch_id"]

ok, res = fetch_batch_output_if_ready(batch_id, BATCH_OUTPUT_JSONL)
if not ok:
    print(f"Batch not complete yet: status={res}  (batch_id={batch_id})")
else:
    meta = json.loads(BATCH_META_JSON.read_text(encoding="utf-8"))

    parsed_records = []
    raw_records    = []
    kept = 0
    blank_answer_count = 0
    one_tag_answer_count = 0

    with open(BATCH_OUTPUT_JSONL, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            obj = json.loads(line)
            if obj.get("error"):
                continue

            content = extract_assistant_content(obj) or ""
            system_prompt, _ = extract_request_messages(obj)

            cid    = obj.get("custom_id")
            mrow   = meta.get(cid, {})
            uid    = mrow.get("uniprot_id")
            seq    = mrow.get("sequence")
            gt_cap = mrow.get("ground_truth")

            # RAW record (as-is)
            raw_records.append({
                "instruction": system_prompt,
                "input": {"uniprot_id": uid, "protein_sequence": seq},
                "output": content,
                "meta": {"ground_truth_caption": gt_cap, "custom_id": cid}
            })

            # PARSED record (loose)
            thinking, answer = parse_thinking_answer_loose(content)
            if "<answer" in content.lower() and "</answer>" not in content.lower():
                one_tag_answer_count += 1
            if thinking and not answer:
                blank_answer_count += 1

            parsed_records.append({
                "instruction": system_prompt,
                "input": {"uniprot_id": uid, "protein_sequence": seq},
                "output": f"<thinking>\n{thinking}\n</thinking>\n\n<answer>\n{answer}\n</answer>",
                "meta": {"ground_truth_caption": gt_cap, "custom_id": cid}
            })
            kept += 1

    SFT_OUT_JSON_PARSED.write_text(json.dumps(parsed_records, ensure_ascii=False, indent=2), encoding="utf-8")
    SFT_OUT_JSON_RAW.write_text(json.dumps(raw_records,    ensure_ascii=False, indent=2), encoding="utf-8")

    print({
        "kept": kept,
        "parsed_out": str(SFT_OUT_JSON_PARSED),
        "raw_out": str(SFT_OUT_JSON_RAW),
        "one_tag_answer_count": one_tag_answer_count,
        "blank_answer_after_parse": blank_answer_count
    })

    # Quick preview
    if parsed_records:
        print("\n--- SAMPLE (parsed) ---")
        print("INSTRUCTION (truncated):", parsed_records[0]["instruction"][:220], "...")
        print("OUTPUT (first 300 chars):", parsed_records[0]["output"][:300], "...")
    if raw_records:
        print("\n--- SAMPLE (raw) ---")
        print("INSTRUCTION (truncated):", raw_records[0]["instruction"][:220], "...")
        print("OUTPUT (first 300 chars):", raw_records[0]["output"][:300], "...")

{'kept': 100, 'parsed_out': '/content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc/protein2desc_cot_sft.json', 'raw_out': '/content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc/protein2desc_cot_raw.json', 'one_tag_answer_count': 77, 'blank_answer_after_parse': 0}

--- SAMPLE (parsed) ---
INSTRUCTION (truncated):  ...
OUTPUT (first 300 chars): <thinking>
1. **Signal Peptide and Transmembrane Regions**: The sequence begins with a hydrophobic segment (MGLFDRGVQMLLTTVGAFAAFSLMTIAVGTDYWLYSRGVCK), suggesting a potential signal peptide or transmembrane domain. The presence of multiple hydrophobic stretches (e.g., LSVILLFMGGLCIAASEFYKTRHNIILSAGI ...

--- SAMPLE (raw) ---
INSTRUCTION (truncated):  ...
OUTPUT (first 300 chars): <thinking>
1. **Signal Peptide and Transmembrane Regions**: The sequence begins with a hydrophobic segment (MGLFDRGVQMLLTTVGAFAAFSLMTIAVGTDYWLYSRGVCK), suggesting a potential signal peptide or transmembrane domain. The p

In [ ]:
# moving files if you set wrong working directory.
import os, shutil, hashlib
from pathlib import Path


SRC  = Path("/content/gpt_batch_protein2desc")
DEST = BASE_DIR / "gpt_batch_protein2desc"

def count_files_and_bytes(root: Path):
    n, b = 0, 0
    for p in root.rglob("*"):
        if p.is_file():
            n += 1
            try:
                b += p.stat().st_size
            except Exception:
                pass
    return n, b

if not SRC.exists():
    raise FileNotFoundError(f"Source folder not found: {SRC}")

DEST.mkdir(parents=True, exist_ok=True)

src_files, src_bytes = count_files_and_bytes(SRC)
print(f"[SRC] {SRC}  files={src_files}  size={src_bytes/1e6:.2f} MB")

if not any(DEST.iterdir()):

    for item in SRC.iterdir():
        shutil.move(str(item), str(DEST))
else:

    for root, dirs, files in os.walk(SRC):
        rel = Path(root).relative_to(SRC)
        dest_dir = DEST / rel
        dest_dir.mkdir(parents=True, exist_ok=True)
        for d in dirs:
            (dest_dir / d).mkdir(parents=True, exist_ok=True)
        for f in files:
            s = Path(root) / f
            t = dest_dir / f
            if t.exists():
                try:
                    t.unlink()
                except IsADirectoryError:
                    shutil.rmtree(t)
            try:
                shutil.move(str(s), str(t))
            except Exception:
                shutil.copy2(str(s), str(t))
                s.unlink()

dest_files, dest_bytes = count_files_and_bytes(DEST)
print(f"[DEST] {DEST}  files={dest_files}  size={dest_bytes/1e6:.2f} MB")

if dest_files >= src_files and dest_bytes >= src_bytes * 0.999:
    try:
        shutil.rmtree(SRC)
        print(f"[OK] Moved and removed source: {SRC}")
    except Exception as e:
        print(f"[WARN] Could not remove source (remove manually if needed): {e}")
else:
    raise RuntimeError("Sanity check failed: destination counts smaller than source; aborting deletion.")

important = [
    "protein2desc_input.jsonl",
    "protein2desc_meta.json",
    "protein2desc_batch_info.json",
    "protein2desc_output.jsonl",
    "protein2desc_cot_sft.json",
]
print("\nKey files now under:", DEST)
for name in important:
    p = DEST / name
    print(" -", p, ("[OK]" if p.exists() else "[missing]"))

[SRC] /content/gpt_batch_protein2desc  files=5  size=9.33 MB
[DEST] /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc  files=5  size=9.33 MB
[OK] Moved and removed source: /content/gpt_batch_protein2desc

Key files now under: /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc
 - /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc/protein2desc_input.jsonl [OK]
 - /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc/protein2desc_meta.json [OK]
 - /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc/protein2desc_batch_info.json [OK]
 - /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc/protein2desc_output.jsonl [OK]
 - /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc/protein2desc_cot_sft.json [OK]


In [ ]:
from pathlib import Path
import json

# STEP 3: Fetch & build SFT
from openai import OpenAI
client = OpenAI()

from pathlib import Path
from types import SimpleNamespace as NS

# load batch info and helpers from previous cell (they're defined in-kernel)
binfo = json.loads(BASE_DIR / Path("/gpt_batch_proteindt/batch_info.json").read_text())
outp  = fetch_batch_output(binfo["batch_id"], BASE_DIR / Path("/gpt_batch_proteindt/batch_output.jsonl"))
stats = build_sft_from_outputs(BASE_DIR /  Path("/gpt_batch_proteindt/batch_output.jsonl"),
                               Path(BASE_DIR / "/gpt_batch_proteindt/batch_meta.json"),
                               Path("/content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/proteindt_consistency_sft2.json"))
print(stats)
print("SFT saved at:", Path("./gpt_batch_proteindt/proteindt_consistency_sft2.json").resolve())

{'kept': 0, 'skipped_disagreement': 0, 'skipped_parsing': 1000}
SFT saved at: /content/gpt_batch_proteindt/proteindt_consistency_sft.json


In [ ]:
#@title Parse an OpenAI Batch output JSONL (from dashboard) into SFT JSON
from pathlib import Path
import json, re

BATCH_OUTPUT_PATH = Path("/content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/subset_1k/batch_result_1k.jsonl")   # <-- change me
OUT_SFT_PATH      = Path("/content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/subset_1k/proteindt_sft1.json")

RE_UID = re.compile(r"UniProt:\s*([^\n\r]+)", re.I)
RE_SEQ = re.compile(r"Sequence\s*\(truncated\):\s*([\s\S]+?)\nCaption", re.I)
RE_CAP = re.compile(r"Caption\s*\(truncated\):\s*([\s\S]+?)\n(?:Is the caption|Is this caption|Think|Answer|$)", re.I)
TAG_THINKING = re.compile(r"<thinking>\s*(.*?)\s*</thinking>", re.I | re.S)
TAG_ANSWER   = re.compile(r"<answer>\s*(.*?)\s*</answer>", re.I | re.S)

def extract_content(obj):
    resp = obj.get("response") or {}
    body = resp.get("body") if isinstance(resp, dict) else None
    src = body if isinstance(body, dict) else resp
    try:
        return src["choices"][0]["message"]["content"]
    except Exception:
        return ""

def extract_request_user_text(obj):
    req = obj.get("request") or {}
    rbody = req.get("body") if isinstance(req, dict) else None
    src = rbody if isinstance(rbody, dict) else req
    try:
        msgs = src["messages"]
    except Exception:
        return ""
    for m in reversed(msgs):
        if m.get("role") == "user":
            return m.get("content", "")
    return ""

def parse_io_from_request(user_text: str):
    uid = RE_UID.search(user_text).group(1).strip() if RE_UID.search(user_text) else None
    seq = RE_SEQ.search(user_text).group(1).strip() if RE_SEQ.search(user_text) else None
    cap = RE_CAP.search(user_text).group(1).strip() if RE_CAP.search(user_text) else None
    return uid, seq, cap

def parse_answer_safe(text: str):
    thinking = (TAG_THINKING.search(text).group(1).strip()
                if TAG_THINKING.search(text) else "")
    answer   = (TAG_ANSWER.search(text).group(1).strip()
                if TAG_ANSWER.search(text) else "")
    if not answer:
        low = text.lower()
        if "inconsistent" in low: answer = "Inconsistent"
        elif "consistent" in low: answer = "Consistent"
        else: answer = "Consistent"
    if not thinking:
        thinking = text.strip()
    return thinking, answer

def build_sft(batch_jsonl: Path, out_json: Path) -> dict:
    kept, parse_errors = 0, 0
    records = []
    with open(batch_jsonl, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                obj = json.loads(line)
            except Exception:
                parse_errors += 1
                continue
            if obj.get("error"):
                continue

            content = extract_content(obj) or ""
            if not content:
                parse_errors += 1
                continue

            user_text = extract_request_user_text(obj) or ""
            uid, seq, cap = parse_io_from_request(user_text)
            uid = uid or "UNKNOWN"; seq = seq or ""; cap = cap or ""

            thinking, answer = parse_answer_safe(content)
            records.append({
                "input": {
                    "uniprot_id": uid,
                    "protein_sequence": seq,
                    "caption": cap
                },
                "output": f"<thinking>\n{thinking}\n</thinking>\n\n<answer>{answer}</answer>"
            })
            kept += 1

    out_json.write_text(json.dumps(records, ensure_ascii=False, indent=2), encoding="utf-8")
    return {"kept": kept, "parse_errors": parse_errors, "out": str(out_json)}

stats = build_sft(BATCH_OUTPUT_PATH, OUT_SFT_PATH)
print(stats)
print("Saved:", OUT_SFT_PATH.resolve())

{'kept': 1000, 'parse_errors': 0, 'out': '/content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/subset_1k/proteindt_sft1.json'}
Saved: /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/subset_1k/proteindt_sft1.json


In [ ]:
# ===================== Fetch output & build SFTs (parsed + raw) — with TAG suffix =====================
from pathlib import Path
import json, re, datetime as dt
from openai import OpenAI

# --- Optional: set your own tag to override auto-detection (e.g., "v1_free_2025-09-25") ---
USER_TAG = None  # e.g., "v1a"; leave as None to auto-pick

# Your project root (folder that contains SwissProtCLAP/, etc.)
# BASE_DIR = Path("/content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT")  # set elsewhere in your notebook

# All artifacts live under BASE_DIR / "gpt_batch_protein2desc"
OUT_DIR            = BASE_DIR / "gpt_batch_protein2desc"
BATCH_INFO_JSON    = OUT_DIR / "protein2desc_batch_info.json"
BATCH_OUTPUT_JSONL = OUT_DIR / "protein2desc_output.jsonl"
BATCH_META_JSON    = OUT_DIR / "protein2desc_meta.json"

client = OpenAI()

def fetch_batch_output_if_ready(batch_id: str, out_jsonl: Path):
    """If batch is complete, download output JSONL; else return current status."""
    b = client.batches.retrieve(batch_id)
    if not getattr(b, "output_file_id", None):
        return False, str(b.status)
    content = client.files.content(b.output_file_id)
    data = content.read()
    out_jsonl.write_bytes(data)
    return True, out_jsonl

def extract_assistant_content(obj) -> str:
    """Robust extractor for Batch lines: response.body.choices[0].message.content."""
    resp = obj.get("response") or {}
    body = resp.get("body") if isinstance(resp, dict) else None
    src  = body if isinstance(body, dict) else resp
    try:
        return src["choices"][0]["message"]["content"]
    except Exception:
        return ""

def extract_request_messages(obj):
    """Return (system_content, user_content) from the *request* body messages for this line."""
    req = obj.get("request") or {}
    rbody = req.get("body") if isinstance(req, dict) else None
    src = rbody if isinstance(rbody, dict) else req
    msgs = src.get("messages", []) if isinstance(src, dict) else []
    sys_txt, usr_txt = None, None
    for m in msgs:
        if sys_txt is None and m.get("role") == "system":
            sys_txt = m.get("content", "")
    for m in reversed(msgs):
        if m.get("role") == "user":
            usr_txt = m.get("content", "")
            break
    return sys_txt or "", usr_txt or ""

# -------- Loose tag parser (handles missing closing tags) --------
def _extract_tag_loose(text: str, tag: str):
    """Return (content, endpos, found) for <tag> ... [</tag>] allowing missing closing tag."""
    if not text:
        return "", 0, False
    open_pat  = re.compile(fr"<{tag}\b[^>]*>", re.I)
    close_pat = re.compile(fr"</{tag}\s*>", re.I)
    m_open = open_pat.search(text)
    if not m_open:
        return "", 0, False
    m_close = close_pat.search(text, m_open.end())
    if m_close:
        content = text[m_open.end():m_close.start()]
        endpos  = m_close.end()
    else:
        # no closing tag → capture to end
        content = text[m_open.end():]
        endpos  = len(text)
    # clean up stray trailing characters often seen in CSV dumps: quotes/commas
    content = content.strip().rstrip('", \t\r\n')
    return content, endpos, True

def parse_thinking_answer_loose(text: str):
    """
    Extract thinking and answer with tolerant rules:
     - If <thinking> found and closed: use between tags.
     - If <thinking> open only: capture until <answer> (if exists) or end.
     - If <answer> found and closed: use between tags.
     - If <answer> open only: capture to end.
     - If neither found: thinking=text, answer=""
    """
    if not text:
        return "", ""
    thinking, th_end, th_found = _extract_tag_loose(text, "thinking")
    remainder = text[th_end:] if th_found else text
    answer, _, an_found = _extract_tag_loose(remainder, "answer")
    if not th_found and an_found:
        m_open_ans = re.search(r"<answer\b[^>]*>", remainder, re.I)
        cutoff = m_open_ans.start() if m_open_ans else len(text)
        pre = text[:cutoff].strip().rstrip('", \t\r\n')
        thinking = pre if pre else thinking
    if not th_found and not an_found:
        thinking = text.strip().rstrip('", \t\r\n')
        answer   = ""
    return thinking, answer

# ---------- tag helpers ----------
def choose_tag(binfo: dict) -> str:
    if USER_TAG:
        return str(USER_TAG)
    if isinstance(binfo, dict) and binfo.get("run_tag"):
        return str(binfo["run_tag"])
    bid = (binfo.get("batch_id") or "") if isinstance(binfo, dict) else ""
    if bid:
        return f"run_{bid[-8:]}"  # short batch id suffix
    # fallback ISO date-time
    return dt.datetime.now().strftime("run_%Y%m%d-%H%M%S")

def unique_path(base: Path) -> Path:
    """Avoid clobbering if same tag reused; append _1, _2, ... if needed."""
    if not base.exists():
        return base
    stem, suf = base.stem, base.suffix
    for i in range(1, 1000):
        cand = base.with_name(f"{stem}_{i}{suf}")
        if not cand.exists():
            return cand
    return base.with_name(f"{stem}_dup{int(dt.datetime.now().timestamp())}{suf}")

# ---------------------------- main ----------------------------
assert BATCH_INFO_JSON.exists(), f"Missing {BATCH_INFO_JSON}"
assert BATCH_META_JSON.exists(), f"Missing {BATCH_META_JSON}"

binfo = json.loads(BATCH_INFO_JSON.read_text())
batch_id = binfo["batch_id"]
TAG = choose_tag(binfo)

# Build tagged output paths
SFT_OUT_JSON_PARSED = unique_path(OUT_DIR / f"protein2desc_cot_sft_{TAG}.json")
SFT_OUT_JSON_RAW    = unique_path(OUT_DIR / f"protein2desc_cot_raw_{TAG}.json")

ok, res = fetch_batch_output_if_ready(batch_id, BATCH_OUTPUT_JSONL)
if not ok:
    print(f"Batch not complete yet: status={res}  (batch_id={batch_id})")
else:
    meta = json.loads(BATCH_META_JSON.read_text(encoding="utf-8"))

    parsed_records = []
    raw_records    = []
    kept = 0
    blank_answer_count = 0
    one_tag_answer_count = 0

    with open(BATCH_OUTPUT_JSONL, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            obj = json.loads(line)
            if obj.get("error"):
                continue

            content = extract_assistant_content(obj) or ""
            system_prompt, _ = extract_request_messages(obj)

            cid    = obj.get("custom_id")
            mrow   = meta.get(cid, {})
            uid    = mrow.get("uniprot_id")
            seq    = mrow.get("sequence")
            gt_cap = mrow.get("ground_truth")

            # RAW record (as-is)
            raw_records.append({
                "instruction": system_prompt,
                "input": {"uniprot_id": uid, "protein_sequence": seq},
                "output": content,
                "meta": {
                    "ground_truth_caption": gt_cap,
                    "custom_id": cid,
                    "tag": TAG
                }
            })

            # PARSED record (loose)
            thinking, answer = parse_thinking_answer_loose(content)
            if "<answer" in content.lower() and "</answer>" not in content.lower():
                one_tag_answer_count += 1
            if thinking and not answer:
                blank_answer_count += 1

            parsed_records.append({
                "instruction": system_prompt,
                "input": {"uniprot_id": uid, "protein_sequence": seq},
                "output": f"<thinking>\n{thinking}\n</thinking>\n\n<answer>\n{answer}\n</answer>",
                "meta": {
                    "ground_truth_caption": gt_cap,
                    "custom_id": cid,
                    "tag": TAG
                }
            })
            kept += 1

    SFT_OUT_JSON_PARSED.write_text(json.dumps(parsed_records, ensure_ascii=False, indent=2), encoding="utf-8")
    SFT_OUT_JSON_RAW.write_text(json.dumps(raw_records,    ensure_ascii=False, indent=2), encoding="utf-8")

    print({
        "tag": TAG,
        "kept": kept,
        "parsed_out": str(SFT_OUT_JSON_PARSED),
        "raw_out": str(SFT_OUT_JSON_RAW),
        "one_tag_answer_count": one_tag_answer_count,
        "blank_answer_after_parse": blank_answer_count
    })

    # Quick preview
    if parsed_records:
        print("\n--- SAMPLE (parsed) ---")
        print("INSTRUCTION (truncated):", parsed_records[0]["instruction"][:220], "...")
        print("OUTPUT (first 300 chars):", parsed_records[0]["output"][:300], "...")
    if raw_records:
        print("\n--- SAMPLE (raw) ---")
        print("INSTRUCTION (truncated):", raw_records[0]["instruction"][:220], "...")
        print("OUTPUT (first 300 chars):", raw_records[0]["output"][:300], "...")

{'tag': 'run_b0778017', 'kept': 100, 'parsed_out': '/content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc/protein2desc_cot_sft_run_b0778017.json', 'raw_out': '/content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc/protein2desc_cot_raw_run_b0778017.json', 'one_tag_answer_count': 64, 'blank_answer_after_parse': 0}

--- SAMPLE (parsed) ---
INSTRUCTION (truncated):  ...
OUTPUT (first 300 chars): <thinking>
1. **Signal Peptide Analysis**: The sequence starts with a hydrophobic region (MGLFDRGVQMLLTTVGAFAAFSLMTIAVGTDYWLYSRG), which suggests the presence of a signal peptide. This indicates that the protein is likely directed to the secretory pathway or inserted into a membrane.

2. **Transmemb ...

--- SAMPLE (raw) ---
INSTRUCTION (truncated):  ...
OUTPUT (first 300 chars): <thinking>
1. **Signal Peptide Analysis**: The sequence starts with a hydrophobic region (MGLFDRGVQMLLTTVGAFAAFSLMTIAVGTDYWLYSRG), which suggests the presence of a signal 

### Generate CoT with ground truth

In [ ]:
# ===================== GPT Batch: Protein → (Thinking Only) given Ground Truth =====================
# Generate CoT-only rationales: provide sequence + ground truth; model outputs ONLY <thinking>…</thinking>.
# -----------------------------------------------------------------------------------------------
import os, json, re, html, random, time, uuid
from pathlib import Path
from typing import Dict, Any, List, Tuple
from openai import OpenAI

# --- Required ---
os.environ.setdefault("OPENAI_API_KEY", "<PUT_YOUR_KEY_HERE>")  # or set in env beforehand
BASE_DIR         = Path("/content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT")  # ProteinDT root
MODEL            = "gpt-4o"          # e.g., "gpt-4o", "o1-mini", "o1-preview"
N_SAMPLES        = 100              # how many proteins to generate
TEMPERATURE      = 0.1
MAX_TOKENS       = 3000              # only thinking; shorter than desc+thinking
BATCH_COMPLETION_WINDOW = "24h"
RANDOM_SEED      = 42

# --- Optional: reuse your subset manifest ---
MANIFEST_PATH    = BASE_DIR / "subset_1k" / "subset_manifest_1k.csv"  # or set to None
MANIFEST_LIMIT   = 0   # 0 = no extra limit

# --- Outputs (this version’s folder) ---
OUT_DIR            = BASE_DIR / "gpt_batch_protein2desc_thinking"     # <— NEW folder for thinking-only
OUT_DIR.mkdir(parents=True, exist_ok=True)
RUN_TAG            = time.strftime("%Y%m%d-%H%M%S")                   # avoid collisions across runs
BATCH_INPUT_JSONL  = OUT_DIR / f"protein2desc_thinking_input_{RUN_TAG}.jsonl"
BATCH_META_JSON    = OUT_DIR / f"protein2desc_thinking_meta_{RUN_TAG}.json"
BATCH_INFO_JSON    = OUT_DIR / f"protein2desc_thinking_batch_info_{RUN_TAG}.json"
BATCH_OUTPUT_JSONL = OUT_DIR / f"protein2desc_thinking_output_{RUN_TAG}.jsonl"  # downloaded later

client = OpenAI()

# ----------------- helpers -----------------
def strip_html(s: str) -> str:
    if not isinstance(s, str): return ""
    s = html.unescape(s)
    return re.sub(r"<[^>]+>", " ", s).replace("\xa0"," ").strip()

def trunc(s: str, n: int) -> str:
    s = s or ""
    return (s[:n] + "…") if len(s) > n else s

def two_line_pairs(path: Path):
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        while True:
            id_line = f.readline()
            if not id_line: break
            val_line = f.readline()
            if not val_line: break
            yield id_line.strip(), val_line.strip()

def load_swissprotclap(base_dir: Path):
    sp = base_dir / "SwissProtCLAP"
    seq_fp, txt_fp = sp / "protein_sequence.txt", sp / "text_sequence.txt"
    assert seq_fp.exists() and txt_fp.exists(), f"Missing SwissProtCLAP under {sp}"
    id2seq = {pid: seq for pid, seq in two_line_pairs(seq_fp)}
    id2cap = {pid: strip_html(txt) for pid, txt in two_line_pairs(txt_fp)}
    ids = [i for i in id2seq.keys() if i in id2cap]
    return ids, id2seq, id2cap

# --------------- Build SYSTEM prompt with one in-prompt example ---------------
random.seed(RANDOM_SEED)
ids, id2seq, id2cap = load_swissprotclap(BASE_DIR)

# optional restriction to manifest subset
restrict_ids = None
if MANIFEST_PATH and MANIFEST_PATH.exists():
    import pandas as pd
    dfm = pd.read_csv(MANIFEST_PATH)
    restrict_ids = [str(x) for x in dfm["uniprot_id"].astype(str).tolist()]
    if MANIFEST_LIMIT and MANIFEST_LIMIT > 0:
        restrict_ids = restrict_ids[:MANIFEST_LIMIT]

pool = restrict_ids if restrict_ids else ids[:]
random.shuffle(pool)

# One real example for style (uses different protein than targets)
ex_uid = random.choice(pool)
ex_seq = id2seq[ex_uid]
ex_gt  = id2cap[ex_uid]

PROTEIN_PROMPT_WITH_EXAMPLE = (
    "You are a professional protein biologist. Your task is to generate a natural, concise, and biologically "
    "accurate description of a protein based **only** on its amino-acid sequence.\n\n"
    "### Example:\n"
    f"Protein UniProt: {ex_uid}\n"
    f"Protein sequence: {trunc(ex_seq, 1200)}\n"
    f"Answer: <answer> {ex_gt} </answer>\n\n"
    "### Now try this:\n"
    "Given ONLY the sequence, first think step by step about plausible features (signal peptides, transmembrane "
    "helices, repeats, low-complexity segments, catalytic motifs/domains, localization signals), then produce a "
    "polished 2–4 sentence description.\n\n"
    "Your final answer **must** be returned in the format:\n"
    "<thinking>\n[Your reasoning steps using sequence-derived evidence]\n</thinking>\n\n"
    "<answer>\n[2–4 sentences; A natural, concise scientific description of the protein]\n</answer>\n"
    #"Do not include database IDs, citations, or organism names unless they follow directly from sequence class."
)

SYSTEM_PROMPT = (
    "You are a professional protein biologist. You will receive:\n"
    " - an amino-acid sequence, and\n"
    " - a ground-truth textual description of the protein.\n\n"
    "Your job is to produce ONLY the chain-of-thought reasoning that justifies why the provided description is plausible "
    "given the sequence. Base your reasoning on sequence-derived evidence: signal peptides, transmembrane helices, "
    "motifs/domains, catalytic residues, repeats/low-complexity, localization signals, etc. If evidence is weak, hedge appropriately.\n\n"
    "STRICT OUTPUT FORMAT:\n"
    " - Return exactly ONE block: <thinking> ... </thinking>\n"
    " - Do NOT include <answer>.\n"
    " - Do NOT copy or restate the ground truth verbatim; refer to it implicitly (e.g., 'the description's claim about TM helices').\n\n"
    "### Example\n"
    f"Protein UniProt: {ex_uid}\n"
    f"Protein sequence: {trunc(ex_seq, 1000)}\n"
    f"Ground truth description: {trunc(ex_gt, 600)}\n\n"
    "<thinking>\n"
    "Step 1: Scan N-terminus for signal peptides; presence/absence supports secretion vs cytosolic localization.\n"
    "Step 2: Hydrophobic segments count/length → consistent with the description’s membrane topology claim.\n"
    "Step 3: Look for enzyme catalytic motifs (e.g., Ser-His-Asp triads) or metal-binding signatures; compare to description.\n"
    "Step 4: Repeats/low complexity or coiled-coil indicators; match to claimed structural/functional roles.\n"
    "Step 5: Summarize how these features collectively support (or qualify) the ground-truth description.\n"
    "</thinking>\n\n"
    "### Now do the same for each input below.\n"
)

def build_messages_thinking_only(uniprot_id: str, seq: str, ground_truth: str) -> list[dict]:
    user = (
        f"Protein UniProt: {uniprot_id}\n"
        f"Protein sequence (truncated to 2000 aa if long):\n{trunc(seq, 2000)}\n\n"
        f"Ground truth description:\n{trunc(ground_truth, 1200)}\n\n"
        "Return ONLY the reasoning as <thinking>…</thinking> (no <answer>)."
    )
    return [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user},
    ]

# --------------- Build JSONL and submit batch ---------------
usable_pool = [x for x in pool if x != ex_uid]
chosen = usable_pool[:N_SAMPLES]

meta = {}
with open(BATCH_INPUT_JSONL, "w", encoding="utf-8") as fout:
    for i, uid in enumerate(chosen):
        seq = id2seq[uid]
        gt  = id2cap[uid]
        custom_id = f"p2d_th_{RUN_TAG}_{i}_{uid}"
        meta[custom_id] = {
            "uniprot_id": uid,
            "sequence": trunc(seq, 2000),
            "ground_truth": gt,
            "example_uid": ex_uid
        }
        body = {
            "model": MODEL,
            "temperature": TEMPERATURE,
            "messages": build_messages_thinking_only(uid, seq, gt),
            "max_tokens": MAX_TOKENS,
        }
        line = {
            "custom_id": custom_id,
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": body,
        }
        fout.write(json.dumps(line, ensure_ascii=False) + "\n")

BATCH_META_JSON.write_text(json.dumps(meta, ensure_ascii=False, indent=2), encoding="utf-8")
print(f"[Example in prompt] UniProt: {ex_uid}")
print(f"Wrote batch input: {BATCH_INPUT_JSONL}")
print(f"Wrote meta map:   {BATCH_META_JSON}")

# Submit batch
with open(BATCH_INPUT_JSONL, "rb") as f:
    input_file = client.files.create(file=f, purpose="batch")

batch = client.batches.create(
    input_file_id=input_file.id,
    endpoint="/v1/chat/completions",
    completion_window=BATCH_COMPLETION_WINDOW,
)

BATCH_INFO_JSON.write_text(json.dumps({
    "batch_id": batch.id,
    "input_file_id": input_file.id,
    "status": batch.status,
    "run_tag": RUN_TAG,
    "example_uid": ex_uid
}, indent=2), encoding="utf-8")

print("\n=== SUBMITTED BATCH ===")
print("batch_id:      ", batch.id)
print("input_file_id: ", input_file.id)
print("status:        ", batch.status)
print(f"(Saved to {BATCH_INFO_JSON})")

print("\nWhen finished, run the next cell to fetch & assemble the SFT files.")

[Example in prompt] UniProt: P03746
Wrote batch input: /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc_thinking/protein2desc_thinking_input_20250925-171219.jsonl
Wrote meta map:   /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc_thinking/protein2desc_thinking_meta_20250925-171219.json

=== SUBMITTED BATCH ===
batch_id:       batch_68d577fa9d488190a1eaf9fda93f0f0e
input_file_id:  file-78MQrC22KsUyjZBwN2is4f
status:         validating
(Saved to /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc_thinking/protein2desc_thinking_batch_info_20250925-171219.json)

When finished, run the next cell to fetch & assemble the SFT files.


In [ ]:
# ===================== GPT Batch: Fixed-Answer (model fills ONLY <thinking>) =====================
# The SYSTEM prompt stays as your PROTEIN_PROMPT_WITH_EXAMPLE.
# The USER message includes the exact <answer> (ground truth) and asks the model to fill ONLY <thinking>.
# -----------------------------------------------------------------------------------------------
import os, json, re, html, random, time
from pathlib import Path
from typing import Dict, Any, List, Tuple
from openai import OpenAI

# --- Required ---
os.environ.setdefault("OPENAI_API_KEY", "<PUT_YOUR_KEY_HERE>")  # or set in env beforehand
BASE_DIR         = Path("")  # ProteinDT root
MODEL            = "gpt-4o"
N_SAMPLES        = 100
TEMPERATURE      = 0.1
MAX_TOKENS       = 10000   # thinking + (copied) answer; thinking is the only part model must generate
BATCH_COMPLETION_WINDOW = "24h"
RANDOM_SEED      = 42

# --- Optional: reuse your 1k subset manifest ---
MANIFEST_PATH    = BASE_DIR / "subset_1k" / "subset_manifest_1k.csv"  # or set to None
MANIFEST_LIMIT   = 0  # 0 = no extra cap

# --- Outputs (new folder for this variant) ---
OUT_DIR            = BASE_DIR / "gpt_batch_protein2desc_fixed_answer"
OUT_DIR.mkdir(parents=True, exist_ok=True)
RUN_TAG            = time.strftime("%Y%m%d-%H%M%S")
BATCH_INPUT_JSONL  = OUT_DIR / f"protein2desc_fixedans_input_{RUN_TAG}.jsonl"
BATCH_META_JSON    = OUT_DIR / f"protein2desc_fixedans_meta_{RUN_TAG}.json"
BATCH_INFO_JSON    = OUT_DIR / f"protein2desc_fixedans_batch_info_{RUN_TAG}.json"

client = OpenAI()

# ---------------- utils ----------------
def strip_html(s: str) -> str:
    if not isinstance(s, str): return ""
    s = html.unescape(s)
    return re.sub(r"<[^>]+>", " ", s).replace("\xa0"," ").strip()

def trunc(s: str, n: int) -> str:
    s = s or ""
    return (s[:n] + "…") if len(s) > n else s

def two_line_pairs(path: Path):
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        while True:
            id_line = f.readline()
            if not id_line: break
            val_line = f.readline()
            if not val_line: break
            yield id_line.strip(), val_line.strip()

def load_swissprotclap(base_dir: Path):
    sp = base_dir / "SwissProtCLAP"
    seq_fp, txt_fp = sp / "protein_sequence.txt", sp / "text_sequence.txt"
    assert seq_fp.exists() and txt_fp.exists(), f"Missing SwissProtCLAP under {sp}"
    id2seq = {pid: seq for pid, seq in two_line_pairs(seq_fp)}
    id2cap = {pid: strip_html(txt) for pid, txt in two_line_pairs(txt_fp)}
    ids = [i for i in id2seq.keys() if i in id2cap]
    return ids, id2seq, id2cap

# ---------------- your SYSTEM prompt (unchanged) ----------------
# Use exactly your string (paste/keep your existing definition here).
def make_system_prompt_with_example(ex_uid: str, ex_seq: str, ex_gt: str) -> str:
    return (
        "You are a professional protein biologist. Your task is to generate a natural, concise, and biologically "
        "accurate description of a protein based **only** on its amino-acid sequence.\n\n"
        "### Example:\n"
        f"Protein UniProt: {ex_uid}\n"
        f"Protein sequence: {trunc(ex_seq, 1200)}\n"
        f"Answer: <answer> {ex_gt} </answer>\n\n"
        "### Now try this:\n"
        "Given ONLY the sequence, first think step by step about plausible features (signal peptides, transmembrane "
        "helices, repeats, low-complexity segments, catalytic motifs/domains, localization signals), then produce a "
        "polished 2–4 sentence description.\n\n"
        "Your final answer **must** be returned in the format:\n"
        "<thinking>\n[steps: Your reasoning steps using sequence-derived evidence]\n</thinking>\n\n"
        "<answer>\n[2–4 sentences; A natural, concise scientific description of the protein]\n</answer>\n"
    )

# --------------- Build messages where USER supplies the exact <answer> ---------------
def build_messages_fill_thinking_fixed_answer(system_prompt: str, uniprot_id: str, seq: str, ground_truth: str):
    """
    We give the exact <answer> block to the model and ask it to generate ONLY the <thinking> block,
    then output both blocks with the answer copied verbatim.
    """
    user = (
        f"Protein UniProt: {uniprot_id}\n"
        f"Protein sequence (truncated to 2000 aa if long):\n{trunc(seq, 2000)}\n\n"
        #"Fill ONLY the <thinking> block using sequence-derived evidence. "
        #"Then output BOTH blocks exactly as below, keeping the <answer> text EXACTLY as provided.\n\n"
        "<thinking>\n[Your reasoning steps using sequence-derived evidence]\n</thinking>\n\n"
        "<answer>\n"
        f"{ground_truth}\n"
        "</answer>"
    )
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user},
    ]

# ---------------- Build & submit batch ----------------
random.seed(RANDOM_SEED)
ids, id2seq, id2cap = load_swissprotclap(BASE_DIR)

# optional restriction to manifest subset
restrict_ids = None
if MANIFEST_PATH and MANIFEST_PATH.exists():
    import pandas as pd
    dfm = pd.read_csv(MANIFEST_PATH)
    restrict_ids = [str(x) for x in dfm["uniprot_id"].astype(str).tolist()]
    if MANIFEST_LIMIT and MANIFEST_LIMIT > 0:
        restrict_ids = restrict_ids[:MANIFEST_LIMIT]

pool = restrict_ids if restrict_ids else ids[:]
random.shuffle(pool)

# Example for system prompt
ex_uid = pool[0]
ex_seq = id2seq[ex_uid]
ex_gt  = id2cap[ex_uid]
SYSTEM_PROMPT_WITH_EXAMPLE = make_system_prompt_with_example(ex_uid, ex_seq, ex_gt)

# Build items (exclude the example ID from targets)
usable = [x for x in pool if x != ex_uid]
chosen = usable[:N_SAMPLES]

meta = {}
with open(BATCH_INPUT_JSONL, "w", encoding="utf-8") as fout:
    for i, uid in enumerate(chosen):
        seq = id2seq[uid]
        gt  = id2cap[uid]  # ground truth that will be FIXED in <answer>
        cid = f"p2d_fixans_{RUN_TAG}_{i}_{uid}"
        meta[cid] = {
            "uniprot_id": uid,
            "sequence": trunc(seq, 2000),
            "ground_truth": gt,
            "example_uid": ex_uid
        }
        body = {
            "model": MODEL,
            "temperature": TEMPERATURE,
            "messages": build_messages_fill_thinking_fixed_answer(SYSTEM_PROMPT_WITH_EXAMPLE, uid, seq, gt),
            "max_tokens": MAX_TOKENS,
        }
        fout.write(json.dumps({
            "custom_id": cid,
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": body
        }, ensure_ascii=False) + "\n")

BATCH_META_JSON.write_text(json.dumps(meta, ensure_ascii=False, indent=2), encoding="utf-8")
print(f"[System example used] UniProt: {ex_uid}")
print(f"Wrote batch input: {BATCH_INPUT_JSONL}")
print(f"Wrote meta map:   {BATCH_META_JSON}")

with open(BATCH_INPUT_JSONL, "rb") as f:
    input_file = client.files.create(file=f, purpose="batch")

batch = client.batches.create(
    input_file_id=input_file.id,
    endpoint="/v1/chat/completions",
    completion_window=BATCH_COMPLETION_WINDOW,
)

BATCH_INFO_JSON.write_text(json.dumps({
    "batch_id": batch.id,
    "input_file_id": input_file.id,
    "status": batch.status,
    "run_tag": RUN_TAG,
    "example_uid": ex_uid
}, indent=2), encoding="utf-8")

print("\n=== SUBMITTED BATCH ===")
print("batch_id:      ", batch.id)
print("input_file_id: ", input_file.id)
print("status:        ", batch.status)
print(f"(Saved to {BATCH_INFO_JSON})")

print("\nWhen the batch finishes, your existing fetch/merge (loose parser) cell will work as-is.")
print("It will recover the real system prompt as `instruction`, and parse <thinking>/<answer> from model outputs.")

[System example used] UniProt: A9M1F6
Wrote batch input: /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc_fixed_answer/protein2desc_fixedans_input_20250925-212446.jsonl
Wrote meta map:   /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc_fixed_answer/protein2desc_fixedans_meta_20250925-212446.json

=== SUBMITTED BATCH ===
batch_id:       batch_68d5b32372048190b56269225499429d
input_file_id:  file-PjazxKv8wTA3dimHQM2Eco
status:         validating
(Saved to /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc_fixed_answer/protein2desc_fixedans_batch_info_20250925-212446.json)

When the batch finishes, your existing fetch/merge (loose parser) cell will work as-is.
It will recover the real system prompt as `instruction`, and parse <thinking>/<answer> from model outputs.


In [ ]:
# ===================== Fetch output & build SFTs (Fixed-Answer variant) =====================
from pathlib import Path
import json, re
from openai import OpenAI

# Base dataset dir (same as in your submit cell)
BASE_DIR = Path("/content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT")  # <- change if needed
OUT_DIR  = BASE_DIR / "gpt_batch_protein2desc_fixed_answer"

# Auto-pick the latest run by timestamp suffix
infos = sorted(OUT_DIR.glob("protein2desc_fixedans_batch_info_20250925-211122.json"))
assert infos, f"No batch_info JSON found under {OUT_DIR}"
BATCH_INFO_JSON = infos[-1]

RUN_TAG = json.loads(BATCH_INFO_JSON.read_text())["run_tag"]
BATCH_OUTPUT_JSONL = OUT_DIR / f"protein2desc_fixedans_output_{RUN_TAG}.jsonl"
BATCH_META_JSON    = OUT_DIR / f"protein2desc_fixedans_meta_{RUN_TAG}.json"

# Output files
SFT_OUT_PARSED = OUT_DIR / f"protein2desc_fixedans_sft_{RUN_TAG}.json"   # parsed + answer forced to GT
SFT_OUT_RAW    = OUT_DIR / f"protein2desc_fixedans_raw_{RUN_TAG}.json"   # raw assistant text

client = OpenAI()

# ---- helpers ----
def fetch_batch_output_if_ready(batch_id: str, out_jsonl: Path):
    b = client.batches.retrieve(batch_id)
    if not getattr(b, "output_file_id", None):
        return False, str(b.status)
    content = client.files.content(b.output_file_id)
    data = content.read()
    out_jsonl.write_bytes(data)
    return True, out_jsonl

def extract_assistant_content(obj) -> str:
    resp = obj.get("response") or {}
    body = resp.get("body") if isinstance(resp, dict) else None
    src  = body if isinstance(body, dict) else resp
    try:
        return src["choices"][0]["message"]["content"]
    except Exception:
        return ""

def extract_system_message(obj) -> str:
    req = obj.get("request") or {}
    rbody = req.get("body") if isinstance(req, dict) else None
    src = rbody if isinstance(rbody, dict) else req
    msgs = src.get("messages", []) if isinstance(src, dict) else []
    for m in msgs:
        if m.get("role") == "system":
            return m.get("content", "")
    return ""

# Loose tag extractors (handle missing closing tags)
def _extract_tag_loose(text: str, tag: str):
    if not text:
        return "", 0, False
    open_pat  = re.compile(fr"<{tag}\b[^>]*>", re.I)
    close_pat = re.compile(fr"</{tag}\s*>", re.I)
    m_open = open_pat.search(text)
    if not m_open:
        return "", 0, False
    m_close = close_pat.search(text, m_open.end())
    if m_close:
        content = text[m_open.end():m_close.start()]
        endpos  = m_close.end()
    else:
        content = text[m_open.end():]
        endpos  = len(text)
    content = content.strip().rstrip('", \t\r\n')
    return content, endpos, True

def parse_thinking_answer_loose(text: str):
    if not text:
        return "", ""
    thinking, th_end, th_found = _extract_tag_loose(text, "thinking")
    remainder = text[th_end:] if th_found else text
    answer, _, an_found = _extract_tag_loose(remainder, "answer")
    if not th_found and an_found:
        m_open_ans = re.search(r"<answer\b[^>]*>", remainder, re.I)
        cutoff = m_open_ans.start() if m_open_ans else len(text)
        pre = text[:cutoff].strip().rstrip('", \t\r\n')
        thinking = pre if pre else thinking
    if not th_found and not an_found:
        thinking = text.strip().rstrip('", \t\r\n')
        answer   = ""
    return thinking, answer

def normalize_text(s: str) -> str:
    s = (s or "").strip()
    s = re.sub(r"\s+", " ", s)
    return s

# ---- run ----
binfo = json.loads(BATCH_INFO_JSON.read_text())
batch_id = binfo["batch_id"]

ok, res = fetch_batch_output_if_ready(batch_id, BATCH_OUTPUT_JSONL)
if not ok:
    print(f"Batch not complete yet: status={res}  (batch_id={batch_id})")
else:
    meta = json.loads(BATCH_META_JSON.read_text(encoding="utf-8"))

    parsed_records = []
    raw_records    = []
    kept = 0
    altered_answers = 0
    missing_thinking_tag = 0
    one_tag_answer = 0

    with open(BATCH_OUTPUT_JSONL, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            obj = json.loads(line)
            if obj.get("error"):
                continue

            content = extract_assistant_content(obj) or ""
            system_prompt = extract_system_message(obj)
            cid  = obj.get("custom_id")
            mrow = meta.get(cid, {})
            uid, seq, gt = mrow.get("uniprot_id"), mrow.get("sequence"), mrow.get("ground_truth")

            # RAW record (no parsing/overrides)
            raw_records.append({
                "instruction": system_prompt,
                "input": {"uniprot_id": uid, "protein_sequence": seq},
                "output": content,
                "meta": {"ground_truth_caption": gt, "custom_id": cid}
            })

            # PARSED record (loose parse; force answer to GT)
            thinking, model_answer = parse_thinking_answer_loose(content)
            if "<thinking" not in content.lower():
                missing_thinking_tag += 1
            if "<answer" in content.lower() and "</answer>" not in content.lower():
                one_tag_answer += 1

            # Compare model's answer to GT (normalize spaces); then FORCE to GT
            model_eq_gt = normalize_text(model_answer) == normalize_text(gt)
            if not model_eq_gt:
                altered_answers += 1

            parsed_records.append({
                "instruction": system_prompt,   # real system prompt as instruction
                "input": {"uniprot_id": uid, "protein_sequence": seq},
                "output": (
                    f"<thinking>\n{thinking}\n</thinking>\n\n"
                    f"<answer>\n{gt}\n</answer>"      # <-- force canonical GT answer
                ),
                "meta": {
                    "ground_truth_caption": gt,
                    "custom_id": cid,
                    "model_answer_equal_to_gt": model_eq_gt
                }
            })
            kept += 1

    # Save both
    SFT_OUT_PARSED.write_text(json.dumps(parsed_records, ensure_ascii=False, indent=2), encoding="utf-8")
    SFT_OUT_RAW.write_text(json.dumps(raw_records,    ensure_ascii=False, indent=2), encoding="utf-8")

    print({
        "run_tag": RUN_TAG,
        "kept": kept,
        "parsed_out": str(SFT_OUT_PARSED),
        "raw_out": str(SFT_OUT_RAW),
        "missing_thinking_tag": missing_thinking_tag,
        "one_tag_answer": one_tag_answer,
        "answer_altered_by_model": altered_answers
    })

    # Quick preview
    if parsed_records:
        print("\n--- SAMPLE (parsed) ---")
        print("INSTRUCTION (truncated):", parsed_records[0]["instruction"][:220], "...")
        print("OUTPUT (first 300 chars):", parsed_records[0]["output"][:300], "...")
    if raw_records:
        print("\n--- SAMPLE (raw) ---")
        print("INSTRUCTION (truncated):", raw_records[0]["instruction"][:220], "...")
        print("OUTPUT (first 300 chars):", raw_records[0]["output"][:300], "...")

{'run_tag': '20250925-211122', 'kept': 100, 'parsed_out': '/content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc_fixed_answer/protein2desc_fixedans_sft_20250925-211122.json', 'raw_out': '/content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc_fixed_answer/protein2desc_fixedans_raw_20250925-211122.json', 'missing_thinking_tag': 0, 'one_tag_answer': 32, 'answer_altered_by_model': 89}

--- SAMPLE (parsed) ---
INSTRUCTION (truncated):  ...
OUTPUT (first 300 chars): <thinking>
1. The sequence begins with "MSEIRKNDHRLMQVLLAPVISEKATLVADKNEQVVFEVAPDATKQEVKAAVELLFKVEVDSVNVLVQKGKQKRFGRSMGRRKDVKKAYVCLKPGQEINFEAEAK", which suggests a lack of a signal peptide, indicating a cytoplasmic localization.
2. The sequence is relatively short, suggesting it might be part of a l ...

--- SAMPLE (raw) ---
INSTRUCTION (truncated):  ...
OUTPUT (first 300 chars): <thinking>
1. The sequence begins with "MSEIRKNDHRLMQVLLAPVISEKATLVADKNEQVVFEVAPDATKQEVKAAVELLFKVEVDS

In [ ]:
# ===================== Fetch output & build SFTs (thinking-only + raw) =====================
from pathlib import Path
import json, re
from openai import OpenAI

#BASE_DIR = Path("/content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT")  # <- change if needed
#OUT_DIR  = BASE_DIR / "gpt_batch_protein2desc_thinking"
OUT_DIR = BASE_DIR / "gpt_batch_protein2desc_fixed_answer"
# Auto-detect the latest run (by timestamp suffix) if multiple exist
cand = sorted(OUT_DIR.glob("protein2desc_thinking_batch_info_*.json"))
assert cand, f"No batch_info JSON found under {OUT_DIR}"
BATCH_INFO_JSON = cand[-1]

RUN_TAG = json.loads(BATCH_INFO_JSON.read_text())["run_tag"]
BATCH_OUTPUT_JSONL = OUT_DIR / f"protein2desc_thinking_output_{RUN_TAG}.jsonl"
BATCH_META_JSON    = OUT_DIR / f"protein2desc_thinking_meta_{RUN_TAG}.json"
SFT_OUT_PARSED     = OUT_DIR / f"protein2desc_thinking_sft_{RUN_TAG}.json"
SFT_OUT_RAW        = OUT_DIR / f"protein2desc_thinking_raw_{RUN_TAG}.json"

client = OpenAI()

def fetch_batch_output_if_ready(batch_id: str, out_jsonl: Path):
    b = client.batches.retrieve(batch_id)
    if not getattr(b, "output_file_id", None):
        return False, str(b.status)
    content = client.files.content(b.output_file_id)
    data = content.read()
    out_jsonl.write_bytes(data)
    return True, out_jsonl

def extract_assistant_content(obj) -> str:
    resp = obj.get("response") or {}
    body = resp.get("body") if isinstance(resp, dict) else None
    src  = body if isinstance(body, dict) else resp
    try:
        return src["choices"][0]["message"]["content"]
    except Exception:
        return ""

def extract_system_message(obj) -> str:
    req = obj.get("request") or {}
    rbody = req.get("body") if isinstance(req, dict) else None
    src = rbody if isinstance(rbody, dict) else req
    msgs = src.get("messages", []) if isinstance(src, dict) else []
    for m in msgs:
        if m.get("role") == "system":
            return m.get("content", "")
    return ""

# Loose extractor for <thinking> only (accepts missing closing tag)
def extract_thinking_loose(text: str) -> str:
    if not text:
        return ""
    open_pat  = re.compile(r"<thinking\b[^>]*>", re.I)
    close_pat = re.compile(r"</thinking\s*>", re.I)
    m_open = open_pat.search(text)
    if not m_open:
        # if no <thinking>, treat full text as thinking (keeps dataset usable)
        return text.strip().rstrip('", \t\r\n')
    m_close = close_pat.search(text, m_open.end())
    if m_close:
        content = text[m_open.end():m_close.start()]
    else:
        # capture till end if closing tag missing
        content = text[m_open.end():]
    return content.strip().rstrip('", \t\r\n')

# ---- run
binfo = json.loads(BATCH_INFO_JSON.read_text())
batch_id = binfo["batch_id"]

ok, res = fetch_batch_output_if_ready(batch_id, BATCH_OUTPUT_JSONL)
if not ok:
    print(f"Batch not complete yet: status={res}  (batch_id={batch_id})")
else:
    meta = json.loads(BATCH_META_JSON.read_text(encoding="utf-8"))
    parsed_records = []
    raw_records    = []
    kept = 0
    missing_tag_count = 0

    with open(BATCH_OUTPUT_JSONL, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            obj = json.loads(line)
            if obj.get("error"):
                continue

            content = extract_assistant_content(obj) or ""
            system_prompt = extract_system_message(obj)
            cid  = obj.get("custom_id")
            mrow = meta.get(cid, {})
            uid, seq, gt = mrow.get("uniprot_id"), mrow.get("sequence"), mrow.get("ground_truth")

            # RAW (no parsing)
            raw_records.append({
                "instruction": system_prompt,
                "input": {
                    "uniprot_id": uid,
                    "protein_sequence": seq,
                    "ground_truth_description": gt
                },
                "output": content,
                "meta": {"custom_id": cid}
            })

            # PARSED (thinking only)
            thinking = extract_thinking_loose(content)
            if "<thinking" not in content.lower():
                missing_tag_count += 1

            parsed_records.append({
                "instruction": system_prompt,  # the real prompt used
                "input": {
                    "uniprot_id": uid,
                    "protein_sequence": seq,
                    "ground_truth_description": gt
                },
                "output": f"<thinking>\n{thinking}\n</thinking>",
                "meta": {"custom_id": cid}
            })
            kept += 1

    SFT_OUT_PARSED.write_text(json.dumps(parsed_records, ensure_ascii=False, indent=2), encoding="utf-8")
    SFT_OUT_RAW.write_text(json.dumps(raw_records,    ensure_ascii=False, indent=2), encoding="utf-8")

    print({
        "run_tag": RUN_TAG,
        "kept": kept,
        "parsed_out": str(SFT_OUT_PARSED),
        "raw_out": str(SFT_OUT_RAW),
        "missing_thinking_tag": missing_tag_count
    })

    # quick preview
    if parsed_records:
        print("\n--- SAMPLE (parsed) ---")
        print("INSTRUCTION (truncated):", parsed_records[0]["instruction"][:220], "...")
        print("OUTPUT (first 300 chars):", parsed_records[0]["output"][:300], "...")
    if raw_records:
        print("\n--- SAMPLE (raw) ---")
        print("INSTRUCTION (truncated):", raw_records[0]["instruction"][:220], "...")
        print("OUTPUT (first 300 chars):", raw_records[0]["output"][:300], "...")

AssertionError: No batch_info JSON found under /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc_fixed_answer

In [ ]:
# ===================== Compare CoT Lengths: Version 1 vs Fixed-Answer =====================
from pathlib import Path
import json, re, statistics as stats
import pandas as pd

# ---- paths (edit BASE_DIR if needed) ----
BASE_DIR = Path("/content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT")

V1_SFT_PATH = BASE_DIR / "gpt_batch_protein2desc" / "protein2desc_cot_sft.json"
FIXED_DIR   = BASE_DIR / "gpt_batch_protein2desc_fixed_answer"

# pick the newest fixed-answer SFT by timestamp in filename
fixed_candidates = sorted(FIXED_DIR.glob("protein2desc_fixedans_sft_*.json"))
assert fixed_candidates, f"No fixed-answer SFT found under {FIXED_DIR}"
FIXED_SFT_PATH = fixed_candidates[-1]

OUT_CSV = BASE_DIR / "gpt_batch_protein2desc" / "cot_length_comparison.csv"

print("V1 SFT:", V1_SFT_PATH)
print("Fixed-Answer SFT (latest):", FIXED_SFT_PATH)

# ---- loose tag extraction (handles missing closers) ----
def _extract_tag_loose(text: str, tag: str):
    if not text:
        return "", 0, False
    open_pat  = re.compile(fr"<{tag}\b[^>]*>", re.I)
    close_pat = re.compile(fr"</{tag}\s*>", re.I)
    m_open = open_pat.search(text)
    if not m_open:
        return "", 0, False
    m_close = close_pat.search(text, m_open.end())
    if m_close:
        content = text[m_open.end():m_close.start()]
        endpos  = m_close.end()
    else:
        content = text[m_open.end():]
        endpos  = len(text)
    content = content.strip().rstrip('", \t\r\n')
    return content, endpos, True

def extract_thinking(text: str) -> str:
    thinking, _, found = _extract_tag_loose(text or "", "thinking")
    if found:
        return thinking
    # fallback: if no <thinking>, treat whole text as "thinking" (rare; keeps analysis consistent)
    return (text or "").strip().rstrip('", \t\r\n')

def load_sft(path: Path):
    data = json.loads(path.read_text(encoding="utf-8"))
    assert isinstance(data, list), f"Expected list at {path}"
    return data

def lengths_for(records, label: str):
    rows = []
    for i, r in enumerate(records):
        out = (r.get("output") or "")
        t = extract_thinking(out)
        char_len = len(t)
        word_len = len(t.split())
        rows.append({
            "set": label,
            "idx": i,
            "uniprot_id": ((r.get("input") or {}).get("uniprot_id") or ""),
            "char_len": char_len,
            "word_len": word_len,
            "empty": (char_len == 0)
        })
    return rows

def summarize(df, label):
    sub = df[df["set"] == label]
    n = len(sub)
    n_empty = int(sub["empty"].sum())
    non_empty = sub[~sub["empty"]]
    def safe_stats(series):
        if len(series) == 0:
            return {"mean": 0, "median": 0, "p10": 0, "p90": 0, "min": 0, "max": 0}
        return {
            "mean": float(series.mean()),
            "median": float(series.median()),
            "p10": float(series.quantile(0.10)),
            "p90": float(series.quantile(0.90)),
            "min": int(series.min()),
            "max": int(series.max()),
        }
    ch_all = safe_stats(sub["char_len"])
    wd_all = safe_stats(sub["word_len"])
    ch_ne  = safe_stats(non_empty["char_len"])
    wd_ne  = safe_stats(non_empty["word_len"])
    return {
        "set": label,
        "count": n,
        "empty_count": n_empty,
        "empty_pct": round(100 * n_empty / n, 2) if n else 0.0,
        "chars_all_mean": round(ch_all["mean"], 2),
        "chars_all_median": ch_all["median"],
        "chars_nonempty_mean": round(ch_ne["mean"], 2),
        "chars_nonempty_median": ch_ne["median"],
        "words_all_mean": round(wd_all["mean"], 2),
        "words_all_median": wd_all["median"],
        "words_nonempty_mean": round(wd_ne["mean"], 2),
        "words_nonempty_median": wd_ne["median"],
        "chars_all_p10": ch_all["p10"],
        "chars_all_p90": ch_all["p90"],
        "words_all_p10": wd_all["p10"],
        "words_all_p90": wd_all["p90"],
        "chars_all_min": ch_all["min"],
        "chars_all_max": ch_all["max"],
        "words_all_min": wd_all["min"],
        "words_all_max": wd_all["max"],
    }

# ---- load, compute, show ----
v1 = load_sft(V1_SFT_PATH)
fx = load_sft(FIXED_SFT_PATH)

rows = []
rows += lengths_for(v1, label="v1_free")
rows += lengths_for(fx, label="fixed_answer")

df = pd.DataFrame(rows)
summary = pd.DataFrame([summarize(df, "v1_free"), summarize(df, "fixed_answer")])

print("\n=== Summary (characters & words) ===")
print(summary.to_string(index=False))

# Save per-item lengths (handy for plotting later)
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(OUT_CSV, index=False)
print("\nSaved per-item lengths to:", OUT_CSV)

V1 SFT: /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc/protein2desc_cot_sft.json
Fixed-Answer SFT (latest): /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc_fixed_answer/protein2desc_fixedans_sft_20250925-204511.json

=== Summary (characters & words) ===
         set  count  empty_count  empty_pct  chars_all_mean  chars_all_median  chars_nonempty_mean  chars_nonempty_median  words_all_mean  words_all_median  words_nonempty_mean  words_nonempty_median  chars_all_p10  chars_all_p90  words_all_p10  words_all_p90  chars_all_min  chars_all_max  words_all_min  words_all_max
     v1_free    100            0        0.0         1562.70            1552.0              1562.70                 1552.0          196.00             201.5               196.00                  201.5         1265.0         1801.4          140.7          235.3            994           2513             87            259
fixed_answer    100            0        0

In [ ]:
# ===================== Compare CoT Lengths across ALL versions/tags =====================
from pathlib import Path
import json, re
import pandas as pd

# ---- base path (edit if needed) ----
BASE_DIR = Path("/content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT")

V1_DIR     = BASE_DIR / "gpt_batch_protein2desc"
FIXED_DIR  = BASE_DIR / "gpt_batch_protein2desc_fixed_answer"

# ---- where to save results ----
OUT_PER_ITEM_CSV = BASE_DIR / "cot_length_comparison_all_versions.csv"
OUT_SUMMARY_CSV  = BASE_DIR / "cot_length_summary_all_versions.csv"

# ---- loose tag extraction (handles missing closers) ----
def _extract_tag_loose(text: str, tag: str):
    if not text:
        return "", 0, False
    open_pat  = re.compile(fr"<{tag}\b[^>]*>", re.I)
    close_pat = re.compile(fr"</{tag}\s*>", re.I)
    m_open = open_pat.search(text)
    if not m_open:
        return "", 0, False
    m_close = close_pat.search(text, m_open.end())
    if m_close:
        content = text[m_open.end():m_close.start()]
        endpos  = m_close.end()
    else:
        # no closing tag → capture to end
        content = text[m_open.end():]
        endpos  = len(text)
    content = content.strip().rstrip('", \t\r\n')
    return content, endpos, True

def extract_thinking(text: str) -> str:
    thinking, _, found = _extract_tag_loose(text or "", "thinking")
    if found:
        return thinking
    # fallback: treat whole text as "thinking"
    return (text or "").strip().rstrip('", \t\r\n')

def load_sft(path: Path):
    data = json.loads(path.read_text(encoding="utf-8"))
    if not isinstance(data, list):
        raise ValueError(f"Expected list at {path}")
    return data

def lengths_for(records, set_label: str, tag: str, src_file: Path):
    rows = []
    for i, r in enumerate(records):
        out = (r.get("output") or "")
        t = extract_thinking(out)
        rows.append({
            "family": set_label,                          # 'v1_free' or 'fixed_answer'
            "tag": tag,                                   # e.g. timestamp or 'default'
            "set": f"{set_label}:{tag}",                  # combined label
            "idx": i,
            "uniprot_id": ((r.get("input") or {}).get("uniprot_id") or ""),
            "char_len": len(t),
            "word_len": len(t.split()),
            "empty": (len(t) == 0),
            "source_file": str(src_file)
        })
    return rows

def summarize_group(df, set_label: str):
    sub = df[df["set"] == set_label]
    n = len(sub)
    n_empty = int(sub["empty"].sum())
    non_empty = sub[~sub["empty"]]
    def stats_series(series):
        if len(series) == 0:
            return {"mean": 0, "median": 0, "p10": 0, "p90": 0, "min": 0, "max": 0}
        return {
            "mean": float(series.mean()),
            "median": float(series.median()),
            "p10": float(series.quantile(0.10)),
            "p90": float(series.quantile(0.90)),
            "min": int(series.min()),
            "max": int(series.max()),
        }
    ch_all = stats_series(sub["char_len"])
    wd_all = stats_series(sub["word_len"])
    ch_ne  = stats_series(non_empty["char_len"])
    wd_ne  = stats_series(non_empty["word_len"])
    return {
        "set": set_label,
        "count": n,
        "empty_count": n_empty,
        "empty_pct": round(100 * n_empty / n, 2) if n else 0.0,
        "chars_all_mean": round(ch_all["mean"], 2),
        "chars_all_median": ch_all["median"],
        "chars_nonempty_mean": round(ch_ne["mean"], 2),
        "chars_nonempty_median": ch_ne["median"],
        "words_all_mean": round(wd_all["mean"], 2),
        "words_all_median": wd_all["median"],
        "words_nonempty_mean": round(wd_ne["mean"], 2),
        "words_nonempty_median": wd_ne["median"],
        "chars_all_p10": ch_all["p10"],
        "chars_all_p90": ch_all["p90"],
        "words_all_p10": wd_all["p10"],
        "words_all_p90": wd_all["p90"],
        "chars_all_min": ch_all["min"],
        "chars_all_max": ch_all["max"],
        "words_all_min": wd_all["min"],
        "words_all_max": wd_all["max"],
    }

# ---- discover all SFT files & tags ----
found_files = []

# v1 variants:
# - protein2desc_cot_sft.json                → tag = 'default'
# - protein2desc_cot_sft_<TAG>.json          → tag = <TAG>
if V1_DIR.exists():
    for p in V1_DIR.glob("protein2desc_cot_sft.json"):
        found_files.append(("v1_free", "default", p))
    for p in V1_DIR.glob("protein2desc_cot_sft_*.json"):
        # capture tag after 'sft_'
        m = re.match(r"protein2desc_cot_sft_(?P<tag>[^.]+)\.json$", p.name)
        tag = m.group("tag") if m else p.stem.replace("protein2desc_cot_sft_", "")
        found_files.append(("v1_free", tag, p))

# fixed-answer variants:
# - protein2desc_fixedans_sft_<TAG>.json     → tag = <TAG>
if FIXED_DIR.exists():
    for p in FIXED_DIR.glob("protein2desc_fixedans_sft_*.json"):
        m = re.match(r"protein2desc_fixedans_sft_(?P<tag>[^.]+)\.json$", p.name)
        if not m:
            continue
        tag = m.group("tag")
        found_files.append(("fixed_answer", tag, p))

if not found_files:
    raise FileNotFoundError("No SFT files found in either folder.")

print("Discovered versions:")
for fam, tag, p in found_files:
    print(f" - [{fam}:{tag}]  {p}")

# ---- load all, compute per-item rows ----
rows = []
for fam, tag, p in found_files:
    try:
        recs = load_sft(p)
        rows += lengths_for(recs, set_label=fam, tag=tag, src_file=p)
    except Exception as e:
        print(f"[WARN] Skipping {p}: {e}")

df = pd.DataFrame(rows)
assert not df.empty, "No records loaded from discovered SFT files."

# ---- build summary for each version (set) ----
sets = sorted(df["set"].unique())
summary_rows = [summarize_group(df, s) for s in sets]
summary_df = pd.DataFrame(summary_rows).sort_values(["set"]).reset_index(drop=True)

print("\n=== CoT Length Summary by Version ===")
print(summary_df.to_string(index=False))

# ---- save outputs ----
OUT_PER_ITEM_CSV.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(OUT_PER_ITEM_CSV, index=False)
summary_df.to_csv(OUT_SUMMARY_CSV, index=False)

print("\nSaved per-item lengths to:", OUT_PER_ITEM_CSV)
print("Saved summary to:", OUT_SUMMARY_CSV)

Discovered versions:
 - [v1_free:default]  /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc/protein2desc_cot_sft.json
 - [v1_free:run_b0778017]  /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc/protein2desc_cot_sft_run_b0778017.json
 - [fixed_answer:20250925-180228]  /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc_fixed_answer/protein2desc_fixedans_sft_20250925-180228.json
 - [fixed_answer:20250925-204511]  /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc_fixed_answer/protein2desc_fixedans_sft_20250925-204511.json
 - [fixed_answer:20250925-212446]  /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc_fixed_answer/protein2desc_fixedans_sft_20250925-212446.json
 - [fixed_answer:20250925-211122]  /content/drive/MyDrive/LLM/Bioreasoner/data/hf/proteinDT/gpt_batch_protein2desc_fixed_answer/protein2desc_fixedans_sft_20250925-211122.json

=== 

In [ ]:
from datasets import load_dataset
# Grab SFT prompts (molecule captioning) from the HF dataset repo
from datasets import load_dataset
from huggingface_hub import list_repo_files

REPO_ID = "guojianz/MolReasoner"

# List all files in the dataset repo
files = list_repo_files(REPO_ID, repo_type="dataset")

# Pick the SFT JSON files (names can vary; these are the ones the README mentions)
# Try to find them robustly:
def find_one(name_part):
    cand = [f for f in files if name_part in f]
    assert cand, f"Could not find file containing: {name_part}"
    return f"hf://datasets/{REPO_ID}/{cand[0]}"

cap_train_json = find_one("molecule_captioning_train.json")              # SFT (SELFIES→text)
cap_test_json  = find_one("molecule_captioning_test.json")
t2s_train_json = find_one("text_based_de_novo_molecule_generation_train.json")  # SFT (text→SELFIES)
t2s_test_json  = find_one("text_based_de_novo_molecule_generation_test.json")

# Load the captioning SFT (has CoT prompts)
cap_train = load_dataset("json", data_files=cap_train_json, split="train")

print("Columns:", cap_train.column_names)
row = cap_train[0]

# The prompt is usually a list of {role, content} messages; handle both shapes safely:
prompt = row.get( 'input')
if isinstance(prompt, list) and prompt and isinstance(prompt[0], dict):
    # OpenAI-style chat messages
    system_msgs = [m["content"] for m in prompt if m.get("role") == "system"]
    print("SYSTEM PROMPT EXCERPT:\n", (system_msgs[0] if system_msgs else "")[:600])
else:
    print("PROMPT (raw):\n", str(prompt)[:600])

sft/molecule_captioning/molecule_caption(…):   0%|          | 0.00/78.7M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Columns: ['instruction', 'input', 'output', 'smiles', 'selfies', 'description']
PROMPT (raw):
 None


In [ ]:

# The prompt is usually a list of {role, content} messages; handle both shapes safely:
prompt = row.get('input')
instruction = row.get('instruction')

print("PROMPT (raw):\n", str(prompt)[:1000])
print("INSTRUCTION (raw):\n", str(instruction)[:1000])

PROMPT (raw):
 [C][C][Branch1][C][C][Branch2][Branch1][=Branch2][C][O][P][=Branch1][C][=O][Branch1][C][O][O][P][=Branch1][C][=O][Branch1][C][O][O][C][C@H1][O][C@@H1][Branch1][P][N][C][=N][C][=C][Branch1][C][N][N][=C][N][=C][Ring1][#Branch1][Ring1][#Branch2][C@H1][Branch1][C][O][C@@H1][Ring1][S][O][P][=Branch1][C][=O][Branch1][C][O][O][C@@H1][Branch1][C][O][C][=Branch1][C][=O][N][C][C][C][=Branch1][C][=O][N][C][C][S][C][=Branch1][C][=O][C][/C][=C][\C][C][=O]
INSTRUCTION (raw):
 You are a professional chemist. Your task is to generate a natural, concise, and chemically accurate description of a given molecule.

Please provide a step-by-step analysis explaining how you interpret the molecular structure, identify key features and functional groups, and summarize it into a clear and informative description.

Think step by step, and your final answer **must** be returned in the format: <answer> ... </answer>. For example:
<answer>The molecule is an epoxy(hydroxy)icosatrienoate that is the co

In [ ]:
# ===================== GPT Batch: Fixed-Answer at Scale (50k split into 50×1k) =====================
# The SYSTEM prompt stays as your PROTEIN_PROMPT_WITH_EXAMPLE-style content.
# For each 1k subset, we submit a batch where the USER message provides the exact <answer> (GT),
# and the model must fill ONLY the <thinking> block (but output both blocks).
# -----------------------------------------------------------------------------------------------
import os, json, re, html, random, time, math
from pathlib import Path
from typing import Dict, Any, List, Tuple
from openai import OpenAI

# ============ CONFIG ============
# Put your key in the environment or runtime secrets.
# os.environ["OPENAI_API_KEY"] = "sk-..."   # if not already set
client = OpenAI()

# Root with SwissProtCLAP/ etc.
BASE_DIR         = Path("")  # <-- change if needed

# Model/Batch
MODEL            = "gpt-4o"
TEMPERATURE      = 0.2
MAX_TOKENS       = 1500          # thinking only (+ provided answer block)
COMPLETION_WINDOW = "24h"        # "24h" or "48h"

# Dataset size & chunking
N_TOTAL          = 50_000        # target total items
CHUNK_SIZE       = 1_000         # items per batch
SHUFFLE_SEED     = 42            # reproducible sampling
N_CHUNKS_LIMIT   = None          # e.g., 2 for a dry run; None = all chunks

# Optional: restrict IDs via a manifest CSV (with column 'uniprot_id'); else use SwissProtCLAP full pool
MANIFEST_PATH    = None          # BASE_DIR / "subset_50k" / "manifest_50k.csv"
MANIFEST_LIMIT   = 0             # 0 = no extra limit

# Output folder (shared across chunks)
OUT_DIR          = BASE_DIR / "gpt_batch_protein2desc_fixed_answer"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# ============ HELPERS ============
def strip_html(s: str) -> str:
    if not isinstance(s, str): return ""
    s = html.unescape(s)
    return re.sub(r"<[^>]+>", " ", s).replace("\xa0", " ").strip()

def trunc(s: str, n: int) -> str:
    s = s or ""
    return (s[:n] + "…") if len(s) > n else s

def two_line_pairs(path: Path):
    """SwissProtCLAP: (ID line, value line) repeating."""
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        while True:
            id_line = f.readline()
            if not id_line: break
            val_line = f.readline()
            if not val_line: break
            yield id_line.strip(), val_line.strip()

def load_swissprotclap(base_dir: Path):
    sp = base_dir / "SwissProtCLAP"
    seq_fp, txt_fp = sp / "protein_sequence.txt", sp / "text_sequence.txt"
    assert seq_fp.exists() and txt_fp.exists(), f"Missing SwissProtCLAP under {sp}"
    id2seq = {pid: seq for pid, seq in two_line_pairs(seq_fp)}
    id2cap = {pid: strip_html(txt) for pid, txt in two_line_pairs(txt_fp)}
    ids = [i for i in id2seq.keys() if i in id2cap]
    return ids, id2seq, id2cap

def make_system_prompt_with_example(ex_uid: str, ex_seq: str, ex_gt: str) -> str:
    """Your original PROTEIN_PROMPT_WITH_EXAMPLE (fixed-answer variant will put the answer in user msg)."""
    return (
        "You are a professional protein biologist. Your task is to generate a natural, concise, and biologically "
        "accurate description of a protein based **only** on its amino-acid sequence.\n\n"
        "### Example:\n"
        f"Protein UniProt: {ex_uid}\n"
        f"Protein sequence: {trunc(ex_seq, 1200)}\n"
        f"Answer: <answer> {ex_gt} </answer>\n\n"
        "### Now try this:\n"
        "Given ONLY the sequence, first think step by step about plausible features (signal peptides, transmembrane "
        "helices, repeats, low-complexity segments, catalytic motifs/domains, localization signals), then produce a "
        "polished 2–4 sentence description.\n\n"
        "Your final answer **must** be returned in the format:\n"
        "<thinking>\n[Your reasoning steps using sequence-derived evidence]\n</thinking>\n\n"
        "<answer>\n[2–4 sentences; A natural, concise scientific description of the protein]\n</answer>\n"
    )

def build_messages_fill_thinking_fixed_answer(system_prompt: str, uniprot_id: str, seq: str, ground_truth: str):
    """
    We provide the exact <answer> (ground truth) and ask model to fill ONLY <thinking>.
    Model should output BOTH blocks, keeping <answer> exactly as given.
    """
    user = (
        f"Protein UniProt: {uniprot_id}\n"
        f"Protein sequence (truncated to 2000 aa if long):\n{trunc(seq, 2000)}\n\n"
        "Fill ONLY the <thinking> block using sequence-derived evidence. "
        "Then output BOTH blocks exactly as below, keeping the <answer> text EXACTLY as provided.\n\n"
        "<thinking>\n[Your reasoning steps using sequence-derived evidence]\n</thinking>\n\n"
        "<answer>\n"
        f"{ground_truth}\n"
        "</answer>"
    )
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user},
    ]

def write_json(path: Path, obj: dict|list):
    path.write_text(json.dumps(obj, ensure_ascii=False, indent=2), encoding="utf-8")

# ============ BUILD ID LIST ============
random.seed(SHUFFLE_SEED)
ids, id2seq, id2cap = load_swissprotclap(BASE_DIR)

# optional manifest restriction
if MANIFEST_PATH and Path(MANIFEST_PATH).exists():
    import pandas as pd
    dfm = pd.read_csv(MANIFEST_PATH)
    restrict_ids = [str(x) for x in dfm["uniprot_id"].astype(str).tolist()]
    if MANIFEST_LIMIT and MANIFEST_LIMIT > 0:
        restrict_ids = restrict_ids[:MANIFEST_LIMIT]
    pool = [i for i in restrict_ids if i in id2seq and i in id2cap]
else:
    pool = ids[:]

random.shuffle(pool)

# Select example for the system prompt
ex_uid = pool[0]
ex_seq = id2seq[ex_uid]
ex_gt  = id2cap[ex_uid]
SYSTEM_PROMPT = make_system_prompt_with_example(ex_uid, ex_seq, ex_gt)

# Build the 50k (or as many as available), excluding the example uid globally
usable = [x for x in pool if x != ex_uid]
if len(usable) < N_TOTAL:
    print(f"[WARN] Pool smaller than requested N_TOTAL={N_TOTAL}. Using {len(usable)}.")
target_ids = usable[:N_TOTAL]

# Chunk plan
n_items   = len(target_ids)
n_chunks  = math.ceil(n_items / CHUNK_SIZE) if n_items else 0
if N_CHUNKS_LIMIT is not None:
    n_chunks = min(n_chunks, int(N_CHUNKS_LIMIT))

print(f"Example UID used in system prompt: {ex_uid}")
print(f"Total selected items: {n_items}  |  CHUNK_SIZE={CHUNK_SIZE}  →  {n_chunks} chunks")

# ============ SUBMIT LOOP ============
submitted = []

for c in range(n_chunks):
    start = c * CHUNK_SIZE
    end   = min((c+1) * CHUNK_SIZE, n_items)
    chunk_ids = target_ids[start:end]
    chunk_tag = time.strftime("%Y%m%d-%H%M%S")
    chunk_pad = f"{c:03d}"

    # File names for this chunk
    BATCH_INPUT_JSONL = OUT_DIR / f"protein2desc_fixedans_input_{chunk_tag}_chunk{chunk_pad}.jsonl"
    BATCH_META_JSON   = OUT_DIR / f"protein2desc_fixedans_meta_{chunk_tag}_chunk{chunk_pad}.json"
    BATCH_INFO_JSON   = OUT_DIR / f"protein2desc_fixedans_batch_info_{chunk_tag}_chunk{chunk_pad}.json"

    # Build JSONL + meta
    meta = {}
    with open(BATCH_INPUT_JSONL, "w", encoding="utf-8") as fout:
        for i, uid in enumerate(chunk_ids):
            seq = id2seq[uid]
            gt  = id2cap[uid]  # ground truth answer
            cid = f"p2d_fixans_{chunk_tag}_c{chunk_pad}_{i:04d}_{uid}"
            meta[cid] = {
                "uniprot_id": uid,
                "sequence": trunc(seq, 2000),
                "ground_truth": gt,
                "example_uid": ex_uid,
                "chunk_index": c,
                "chunk_size": len(chunk_ids)
            }
            body = {
                "model": MODEL,
                "temperature": TEMPERATURE,
                "messages": build_messages_fill_thinking_fixed_answer(SYSTEM_PROMPT, uid, seq, gt),
                "max_tokens": MAX_TOKENS,
            }
            fout.write(json.dumps({
                "custom_id": cid,
                "method": "POST",
                "url": "/v1/chat/completions",
                "body": body
            }, ensure_ascii=False) + "\n")

    write_json(BATCH_META_JSON, meta)

    # Submit the chunk as a batch
    with open(BATCH_INPUT_JSONL, "rb") as f:
        input_file = client.files.create(file=f, purpose="batch")

    batch = client.batches.create(
        input_file_id=input_file.id,
        endpoint="/v1/chat/completions",
        completion_window=COMPLETION_WINDOW,
    )

    write_json(BATCH_INFO_JSON, {
        "batch_id": batch.id,
        "input_file_id": input_file.id,
        "status": batch.status,
        "run_tag": chunk_tag,
        "chunk_index": c,
        "chunk_size": len(chunk_ids),
        "n_total_planned": n_items,
        "chunk_file_prefix": f"protein2desc_fixedans_*_{chunk_tag}_chunk{chunk_pad}.*"
    })

    submitted.append({
        "chunk": c,
        "count": len(chunk_ids),
        "batch_id": batch.id,
        "status": batch.status,
        "info_json": str(BATCH_INFO_JSON)
    })

    # (Optional) throttle between submissions if you want to space them out a bit
    # time.sleep(1)

print("\n=== SUBMITTED CHUNKS ===")
for s in submitted:
    print(f"chunk={s['chunk']:03d}  items={s['count']:4d}  batch_id={s['batch_id']}  status={s['status']}")
    print(f"  -> info: {s['info_json']}")